In [1]:
!pip install selenium webdriver-manager pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import random

class IndeedSeleniumScraper:
    def __init__(self, headless=True):
        self.options = Options()
        if headless:
            self.options.add_argument('--headless')
        self.options.add_argument('--no-sandbox')
        self.options.add_argument('--disable-dev-shm-usage')
        self.options.add_argument('--disable-blink-features=AutomationControlled')
        self.options.add_experimental_option("excludeSwitches", ["enable-automation"])
        self.options.add_experimental_option('useAutomationExtension', False)
        self.options.add_argument('--disable-extensions')
        self.options.add_argument('--disable-gpu')
        self.options.add_argument('--window-size=1920,1080')
        
        # Real user agent
        self.options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
        
        self.driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()),
            options=self.options
        )
        
        # Hide automation detection
        self.driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        
        self.jobs_data = []
    
    def scrape_jobs(self, job_title="software developer", location="Toronto, ON", max_pages=3):
        """Scrape jobs using Selenium"""
        
        try:
            for page in range(max_pages):
                start_param = page * 10
                if page == 0:
                    url = f"https://ca.indeed.com/jobs?q={job_title.replace(' ', '+')}&l={location.replace(' ', '+').replace(',', '%2C')}"
                else:
                    url = f"https://ca.indeed.com/jobs?q={job_title.replace(' ', '+')}&l={location.replace(' ', '+').replace(',', '%2C')}&start={start_param}"
                
                print(f"Navigating to: {url}")
                self.driver.get(url)
                
                # Wait for page to load
                time.sleep(random.uniform(3, 5))
                
                # Check for CAPTCHA
                if self._check_captcha():
                    print("CAPTCHA detected! Please solve it manually or try again later.")
                    break
                
                # Wait for job cards to load
                try:
                    WebDriverWait(self.driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, '.job_seen_beacon, .cardOutline, .jobsearch-SerpJobCard'))
                    )
                except:
                    print("Job cards not found, trying to continue...")
                
                # Extract jobs from current page
                self._extract_page_jobs()
                
                print(f"Page {page + 1}/{max_pages} completed. Found {len(self.jobs_data)} jobs so far.")
                
                # Random delay between pages
                time.sleep(random.uniform(2, 4))
                
        except Exception as e:
            print(f"Error during scraping: {e}")
        
        finally:
            self.driver.quit()
        
        return self.jobs_data
    
    def _check_captcha(self):
        """Check if CAPTCHA is present"""
        try:
            captcha_elements = self.driver.find_elements(By.ID, 'captcha')
            captcha_input = self.driver.find_elements(By.ID, 'captcha-input')
            return len(captcha_elements) > 0 or len(captcha_input) > 0
        except:
            return False
    
    def _extract_page_jobs(self):
        """Extract jobs from the current page"""
        try:
            # Try multiple selectors for job cards
            selectors = [
                'div.job_seen_beacon',
                'div.cardOutline',
                'div.jobsearch-SerpJobCard',
                'div[data-jk]'
            ]
            
            job_cards = []
            for selector in selectors:
                try:
                    job_cards = self.driver.find_elements(By.CSS_SELECTOR, selector)
                    if job_cards:
                        break
                except:
                    continue
            
            print(f"Found {len(job_cards)} job cards on this page")
            
            for card in job_cards:
                job_info = self._extract_job_info(card)
                if job_info:
                    self.jobs_data.append(job_info)
                    
        except Exception as e:
            print(f"Error extracting jobs from page: {e}")
    
    def _extract_job_info(self, card):
        """Extract information from a single job card"""
        try:
            # Title
            try:
                title_elem = card.find_element(By.CSS_SELECTOR, 'h2.jobTitle, h2.title, a.jcs-JobTitle')
                title = title_elem.text.strip()
            except:
                title = "N/A"
            
            # Company
            try:
                company_elem = card.find_element(By.CSS_SELECTOR, 'span.companyName, span.company')
                company = company_elem.text.strip()
            except:
                company = "N/A"
            
            # Location
            try:
                location_elem = card.find_element(By.CSS_SELECTOR, 'div.companyLocation, div.location')
                location = location_elem.text.strip()
            except:
                location = "N/A"
            
            # Salary
            try:
                salary_elem = card.find_element(By.CSS_SELECTOR, 'div.salary-snippet, span.salaryText')
                salary = salary_elem.text.strip()
            except:
                salary = "Not specified"
            
            # Link
            try:
                link_elem = card.find_element(By.CSS_SELECTOR, 'a.jcs-JobTitle, a.jobtitle')
                link = link_elem.get_attribute('href')
                if link and link.startswith('/'):
                    link = 'https://ca.indeed.com' + link
            except:
                link = "N/A"
            
            # Date
            try:
                date_elem = card.find_element(By.CSS_SELECTOR, 'span.date, span.datePosted')
                date_posted = date_elem.text.strip()
            except:
                date_posted = "N/A"
            
            job_info = {
                'title': title,
                'company': company,
                'location': location,
                'salary': salary,
                'link': link,
                'date_posted': date_posted,
                'scraped_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
            }
            
            return job_info
            
        except Exception as e:
            print(f"Error extracting job info: {e}")
            return None
    
    def save_to_csv(self, filename="indeed_jobs_selenium.csv"):
        """Save results to CSV"""
        if not self.jobs_data:
            print("No jobs data to save.")
            return False
        
        df = pd.DataFrame(self.jobs_data)
        df.to_csv(filename, index=False, encoding='utf-8')
        print(f"Successfully saved {len(self.jobs_data)} jobs to {filename}")
        return True

# Main execution
if __name__ == "__main__":
    print("Starting Indeed Canada Job Scraper...")
    
    # Initialize scraper (set headless=False to see the browser)
    scraper = IndeedSeleniumScraper(headless=False)  # Set to True to run in background
    
    # Scrape jobs
    jobs = scraper.scrape_jobs(
        job_title="",
        location="Canada",
        max_pages=2  # Start with 2 pages
    )
    
    # Save results
    if jobs:
        success = scraper.save_to_csv("canada_developer_jobs.csv")
        if success:
            print("\nFirst 5 job listings:")
            for i, job in enumerate(jobs[:5], 1):
                print(f"\n{i}. {job['title']}")
                print(f"   Company: {job['company']}")
                print(f"   Location: {job['location']}")
                print(f"   Salary: {job['salary']}")
    else:
        print("No jobs were scraped. Please check:")
        print("1. Your internet connection")
        print("2. If Indeed is blocking requests")
        print("3. Try running with headless=False to see what's happening")

Starting Indeed Canada Job Scraper...
Navigating to: https://ca.indeed.com/jobs?q=&l=Canada
Found 15 job cards on this page
Page 1/2 completed. Found 15 jobs so far.
Navigating to: https://ca.indeed.com/jobs?q=&l=Canada&start=10
Job cards not found, trying to continue...
Found 0 job cards on this page
Page 2/2 completed. Found 15 jobs so far.
Successfully saved 15 jobs to canada_developer_jobs.csv

First 5 job listings:

1. Online ESL Tutors for Young Kids
   Company: N/A
   Location: N/A
   Salary: Not specified

2. Destination Specialist
   Company: N/A
   Location: N/A
   Salary: Not specified

3. Concierge (Entretien ménager)
   Company: N/A
   Location: N/A
   Salary: Not specified

4. Armed Guard
   Company: N/A
   Location: N/A
   Salary: Not specified

5. Journalier
   Company: N/A
   Location: N/A
   Salary: Not specified


In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import random
import re

class IndeedCanadaScraper:
    def __init__(self, headless=False):
        self.options = Options()
        
        self.options.add_argument('--no-sandbox')
        self.options.add_argument('--disable-dev-shm-usage')
        self.options.add_argument('--window-size=1920,1080')
        self.options.add_argument('--disable-blink-features=AutomationControlled')
        
        self.options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
        
        if headless:
            self.options.add_argument('--headless')
        
        self.driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()),
            options=self.options
        )
        
        self.jobs_data = []
    
    def scrape_jobs(self, job_title="developer", location="Canada", max_pages=2):
        """Scrape jobs with precise extraction based on observed structure"""
        
        try:
            for page in range(max_pages):
                if page == 0:
                    url = f"https://ca.indeed.com/jobs?q={job_title.replace(' ', '+')}&l={location.replace(' ', '+').replace(',', '%2C')}"
                else:
                    url = f"https://ca.indeed.com/jobs?q={job_title.replace(' ', '+')}&l={location.replace(' ', '+').replace(',', '%2C')}&start={page * 10}"
                
                print(f"\n=== PAGE {page + 1} ===")
                print(f"Navigating to: {url}")
                
                self.driver.get(url)
                time.sleep(4)
                
                try:
                    WebDriverWait(self.driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.job_seen_beacon'))
                    )
                except:
                    print("Job cards not found, continuing anyway...")
                
                jobs_count_before = len(self.jobs_data)
                self._extract_page_jobs_precise()
                jobs_added = len(self.jobs_data) - jobs_count_before
                
                print(f"Added {jobs_added} jobs from page {page + 1}. Total: {len(self.jobs_data)}")
                
                if page < max_pages - 1:
                    time.sleep(random.uniform(2, 4))
                    
        except Exception as e:
            print(f"Error during scraping: {e}")
            import traceback
            traceback.print_exc()
        
        finally:
            self.driver.quit()
        
        return self.jobs_data

    def _extract_page_jobs_precise(self):
        """Extract jobs with precise logic based on observed structure"""
        try:
            job_cards = self.driver.find_elements(By.CSS_SELECTOR, 'div.job_seen_beacon')
            print(f"Found {len(job_cards)} job cards")
            
            for i, card in enumerate(job_cards):
                job_info = self._extract_job_info_precise(card)
                if job_info:
                    self.jobs_data.append(job_info)
                    print(f"  ✓ {job_info['title'][:40]}... | {job_info['company']} | {job_info['location']}")
                else:
                    print(f"  ✗ Failed to extract job {i+1}")
                    
        except Exception as e:
            print(f"Error extracting jobs from page: {e}")

    def _extract_job_info_precise(self, card):
        """Precise job extraction based on the observed structure"""
        try:
            # Get all text content
            card_text = card.text
            lines = [line.strip() for line in card_text.split('\n') if line.strip()]
            
            if not lines:
                return None
            
            # Extract components based on the observed pattern:
            # Pattern from analysis:
            # 0: 'Full Stack .Net Developer'      <- Title
            # 1: 'New'                            <- Status (ignore)
            # 2: 'Urgently hiring'                <- Hiring status (ignore)
            # 3: 'KAnand Corporation'             <- Company
            # 4: 'Remote'                         <- Location
            # 5: '$80–$100 an hour'               <- Salary
            # 6+: Other info
            
            title = self._extract_title_precise(card, lines)
            company = self._extract_company_precise(lines)
            location = self._extract_location_precise(lines)
            salary = self._extract_salary_precise(lines)
            link = self._extract_link_precise(card)
            
            if title and title != "N/A":
                return {
                    'title': title,
                    'company': company,
                    'location': location,
                    'salary': salary,
                    'link': link,
                    'scraped_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
                }
                
        except Exception as e:
            print(f"Error extracting job info: {e}")
        
        return None

    def _extract_title_precise(self, card, lines):
        """Extract title - always first line"""
        try:
            # Title is always the first line after cleaning
            title_elem = card.find_element(By.CSS_SELECTOR, 'h2.jobTitle')
            title = title_elem.text.strip()
            # Remove "New" prefix if present
            title = re.sub(r'^New\s*', '', title)
            return title
        except:
            # Fallback: use first line
            return lines[0] if lines else "N/A"

    def _extract_company_precise(self, lines):
        """Extract company based on observed pattern"""
        # Based on analysis, company appears at different positions
        # Common patterns observed:
        # - After "Urgently hiring" or "New"
        # - Sometimes has rating numbers (3.2, 3.8, etc.)
        # - Sometimes is the actual company name
        
        for i, line in enumerate(lines):
            line_lower = line.lower()
            
            # Skip obvious non-company lines
            if any(word in line_lower for word in ['new', 'urgently hiring', 'just posted', 'today', 'active']):
                continue
                
            # If we find "Urgently hiring", company is usually the next line
            if 'urgently hiring' in line_lower and i + 1 < len(lines):
                return lines[i + 1]
            
            # If we find a rating pattern (like "3.2"), company might be nearby
            if re.match(r'^\d+\.\d+$', line):
                if i + 1 < len(lines):
                    return lines[i + 1]  # Company after rating
                elif i - 1 >= 0:
                    return lines[i - 1]  # Company before rating
            
            # Company names are usually not the title and not locations
            if (i > 0 and  # Not the first line (title)
                len(line) > 2 and len(line) < 60 and  # Reasonable length
                not any(province in line.upper() for province in ['AB', 'BC', 'MB', 'NB', 'NL', 'NS', 'NT', 'NU', 'ON', 'PE', 'QC', 'SK', 'YT']) and  # Not location
                not re.search(r'\$[\d,]+', line) and  # Not salary
                not any(word in line_lower for word in ['remote', 'hybrid', 'full-time', 'part-time', 'temporary', 'contract']) and  # Not job type
                ',' not in line):  # Not location with comma
                return line
        
        return "Company not found"

    def _extract_location_precise(self, lines):
        """Extract location based on observed pattern"""
        canadian_provinces = ['AB', 'BC', 'MB', 'NB', 'NL', 'NS', 'NT', 'NU', 'ON', 'PE', 'QC', 'SK', 'YT']
        
        for line in lines:
            line_upper = line.upper()
            line_lower = line.lower()
            
            # Direct location indicators
            if any(word in line_lower for word in ['remote', 'hybrid', 'on-site']):
                return line
                
            # Canadian province codes
            if any(province in line_upper for province in canadian_provinces):
                return line
                
            # City, Province pattern
            if re.search(r'.*,\s*(AB|BC|MB|NB|NL|NS|NT|NU|ON|PE|QC|SK|YT)', line, re.IGNORECASE):
                return line
        
        # Try to find location by elimination
        for line in lines:
            if (len(line) < 30 and 
                not any(word in line.lower() for word in ['new', 'urgently', 'hiring', 'apply', 'easily']) and
                not re.search(r'\$[\d,]+', line) and
                not re.match(r'^\d+\.\d+$', line)):
                # This might be a location
                return line
                
        return "Location not specified"

    def _extract_salary_precise(self, lines):
        """Extract salary information"""
        salary_patterns = [
            r'\$\d{1,3}(?:,\d{3})*\s*(?:a year|an hour|a month|a week)',
            r'\$\d{1,3}(?:,\d{3})*\s*-\s*\$\d{1,3}(?:,\d{3})*\s*(?:a year|an hour|a month|a week)',
            r'\$\d+(?:\.\d+)?\s*-\s*\$\d+(?:\.\d+)?\s*(?:a year|an hour|a month|a week)',
            r'From \$\d{1,3}(?:,\d{3})*\s*(?:a year|an hour|a month|a week)'
        ]
        
        for line in lines:
            for pattern in salary_patterns:
                if re.search(pattern, line, re.IGNORECASE):
                    return line
        
        return "Salary not specified"

    def _extract_link_precise(self, card):
        """Extract job link"""
        try:
            link_elem = card.find_element(By.CSS_SELECTOR, 'h2.jobTitle a, a.jcs-JobTitle')
            link = link_elem.get_attribute('href')
            if link:
                if link.startswith('/'):
                    return 'https://ca.indeed.com' + link
                return link
        except:
            pass
            
        try:
            job_id_elem = card.find_element(By.CSS_SELECTOR, 'a[data-jk]')
            job_id = job_id_elem.get_attribute('data-jk')
            if job_id:
                return f'https://ca.indeed.com/viewjob?jk={job_id}'
        except:
            pass
            
        return "Link not available"

    def save_to_csv(self, filename="indeed_canada_jobs_final.csv"):
        """Save results to CSV"""
        if not self.jobs_data:
            print("No jobs data to save.")
            return False
        
        df = pd.DataFrame(self.jobs_data)
        df.to_csv(filename, index=False, encoding='utf-8')
        print(f"Successfully saved {len(self.jobs_data)} jobs to {filename}")
        return True

# Alternative: Use CSS selectors for more reliable extraction
def scrape_with_css_selectors():
    """Try using specific CSS selectors for more reliable extraction"""
    options = Options()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1920,1080')
    
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    
    jobs_data = []
    
    try:
        url = "https://ca.indeed.com/jobs?q=developer&l=Canada"
        print(f"Navigating to: {url}")
        driver.get(url)
        time.sleep(4)
        
        job_cards = driver.find_elements(By.CSS_SELECTOR, 'div.job_seen_beacon')
        print(f"Found {len(job_cards)} job cards")
        
        for i, card in enumerate(job_cards):
            try:
                # Title - reliable
                title_elem = card.find_element(By.CSS_SELECTOR, 'h2.jobTitle')
                title = title_elem.text.strip()
                title = re.sub(r'^New\s*', '', title)
                
                # Company - try multiple selectors
                company = "Company not found"
                company_selectors = [
                    '[data-testid="company-name"]',
                    '[class*="companyName"]',
                    '[class*="company"]'
                ]
                for selector in company_selectors:
                    try:
                        company_elem = card.find_element(By.CSS_SELECTOR, selector)
                        company = company_elem.text.strip()
                        if company:
                            break
                    except:
                        continue
                
                # Location - try multiple selectors
                location = "Location not specified"
                location_selectors = [
                    '[data-testid="text-location"]',
                    '[class*="location"]',
                    '[class*="companyLocation"]'
                ]
                for selector in location_selectors:
                    try:
                        location_elem = card.find_element(By.CSS_SELECTOR, selector)
                        location = location_elem.text.strip()
                        if location:
                            break
                    except:
                        continue
                
                # Salary
                salary = "Salary not specified"
                salary_selectors = [
                    '[data-testid="attribute_snippet_testid"]',
                    '[class*="salary"]',
                    '[class*="compensation"]'
                ]
                for selector in salary_selectors:
                    try:
                        salary_elem = card.find_element(By.CSS_SELECTOR, selector)
                        salary_text = salary_elem.text.strip()
                        if salary_text and '$' in salary_text:
                            salary = salary_text
                            break
                    except:
                        continue
                
                # Link
                link = "Link not available"
                try:
                    link_elem = card.find_element(By.CSS_SELECTOR, 'h2.jobTitle a')
                    link = link_elem.get_attribute('href')
                    if link and link.startswith('/'):
                        link = 'https://ca.indeed.com' + link
                except:
                    pass
                
                job_info = {
                    'title': title,
                    'company': company,
                    'location': location,
                    'salary': salary,
                    'link': link,
                    'scraped_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
                }
                
                jobs_data.append(job_info)
                print(f"{i+1}. {title[:40]}... | {company} | {location}")
                
            except Exception as e:
                print(f"Error with card {i+1}: {e}")
        
        return jobs_data
        
    finally:
        driver.quit()

if __name__ == "__main__":
    print("Starting Final Indeed Canada Job Scraper...")
    
    # Try the CSS selector approach first
    print("=== TRYING CSS SELECTOR APPROACH ===")
    css_jobs = scrape_with_css_selectors()
    
    if css_jobs:
        df = pd.DataFrame(css_jobs)
        df.to_csv("css_selector_jobs.csv", index=False)
        print(f"CSS approach saved {len(css_jobs)} jobs")
        
        print("\nCSS Selector Results:")
        for i, job in enumerate(css_jobs[:3], 1):
            print(f"{i}. {job['title']}")
            print(f"   Company: {job['company']}")
            print(f"   Location: {job['location']}")
            print(f"   Salary: {job['salary']}")
            print()
    
    print("\n=== TRYING PRECISE TEXT ANALYSIS APPROACH ===")
    scraper = IndeedCanadaScraper(headless=False)
    jobs = scraper.scrape_jobs(
        job_title="developer",
        location="Canada",
        max_pages=1
    )
    
    if jobs:
        scraper.save_to_csv("precise_analysis_jobs.csv")
        print(f"\nPrecise analysis saved {len(jobs)} jobs")
        
        print("\nPrecise Analysis Results:")
        for i, job in enumerate(jobs[:3], 1):
            print(f"{i}. {job['title']}")
            print(f"   Company: {job['company']}")
            print(f"   Location: {job['location']}")
            print(f"   Salary: {job['salary']}")
            print()

Starting Final Indeed Canada Job Scraper...
=== TRYING CSS SELECTOR APPROACH ===
Navigating to: https://ca.indeed.com/jobs?q=developer&l=Canada
Found 16 job cards
1. Full Stack .Net Developer... | KAnand Corporation | Remote
2. Business Developer (Mandarin/Cantonese/V... | UNI-ONE FOOD GROUP INC. | Calgary, AB
3. Wireless DSP Software Developer... | Skycope C-UAS Technologies Inc | Burnaby, BC V5G 1J9
4. Software Engineer - Applications... | CanCap Management Inc. | Hybrid work in Toronto, ON M5E 1M2
5. Robot Programmer... | TransCanada Turbines | Rocky View County, AB
6. Sr Embedded Developer - Security Special... | Dormakaba Group | Montréal, QC
7. Business Developer - Canada... | Roullier | Ontario
8. Senior Geophysical Java Software Enginee... | Halliburton | Calgary, AB T2P 3V4
9. Shopify Developer... | Summit Tools | Burnaby, BC
10. Frontend AI Engineer (Remote - EST Time ... | Funded.club | Toronto, ON
11. ... |  | 
12. Associate, Software Engineer, New Grad... | Capital One - C

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import random
import re

class IndeedCanadaScraper:
    def __init__(self, headless=True):
        self.options = Options()
        
        self.options.add_argument('--no-sandbox')
        self.options.add_argument('--disable-dev-shm-usage')
        self.options.add_argument('--window-size=1920,1080')
        self.options.add_argument('--disable-blink-features=AutomationControlled')
        
        self.options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
        
        if headless:
            self.options.add_argument('--headless')
        
        self.driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()),
            options=self.options
        )
        
        self.jobs_data = []
    
    def scrape_jobs(self, job_title="developer", location="Canada", max_pages=3):
        """Scrape jobs using reliable CSS selectors"""
        
        try:
            for page in range(max_pages):
                if page == 0:
                    url = f"https://ca.indeed.com/jobs?q={job_title.replace(' ', '+')}&l={location.replace(' ', '+').replace(',', '%2C')}"
                else:
                    url = f"https://ca.indeed.com/jobs?q={job_title.replace(' ', '+')}&l={location.replace(' ', '+').replace(',', '%2C')}&start={page * 10}"
                
                print(f"Scraping page {page + 1}...")
                
                self.driver.get(url)
                time.sleep(random.uniform(3, 5))
                
                # Wait for job cards
                try:
                    WebDriverWait(self.driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.job_seen_beacon'))
                    )
                except:
                    print(f"No job cards found on page {page + 1}")
                    continue
                
                jobs_count_before = len(self.jobs_data)
                self._extract_page_jobs_css()
                jobs_added = len(self.jobs_data) - jobs_count_before
                
                print(f"Added {jobs_added} jobs from page {page + 1}")
                
                if page < max_pages - 1:
                    time.sleep(random.uniform(2, 4))
                    
        except Exception as e:
            print(f"Error during scraping: {e}")
        
        finally:
            self.driver.quit()
        
        return self.jobs_data

    def _extract_page_jobs_css(self):
        """Extract jobs using reliable CSS selectors"""
        try:
            job_cards = self.driver.find_elements(By.CSS_SELECTOR, 'div.job_seen_beacon')
            print(f"Found {len(job_cards)} job cards")
            
            for card in job_cards:
                job_info = self._extract_job_info_css(card)
                if job_info:
                    self.jobs_data.append(job_info)
                    
        except Exception as e:
            print(f"Error extracting jobs: {e}")

    def _extract_job_info_css(self, card):
        """Extract job information using CSS selectors"""
        try:
            # Title
            title = self._safe_extract(card, 'h2.jobTitle', 'text')
            if title:
                title = re.sub(r'^New\s*', '', title).strip()
            
            # Company - try multiple selectors
            company = self._safe_extract(card, '[data-testid="company-name"]', 'text')
            if not company:
                company = self._safe_extract(card, '[class*="companyName"]', 'text')
            if not company:
                company = self._safe_extract(card, '[class*="company"]', 'text')
            
            # Location - try multiple selectors
            location = self._safe_extract(card, '[data-testid="text-location"]', 'text')
            if not location:
                location = self._safe_extract(card, '[class*="location"]', 'text')
            if not location:
                location = self._safe_extract(card, '[class*="companyLocation"]', 'text')
            
            # Salary
            salary = "Salary not specified"
            salary_selectors = [
                '[data-testid="attribute_snippet_testid"]',
                '[class*="salary"]',
                '[class*="compensation"]',
                '.salary-snippet'
            ]
            for selector in salary_selectors:
                salary_text = self._safe_extract(card, selector, 'text')
                if salary_text and '$' in salary_text:
                    salary = salary_text
                    break
            
            # Link
            link = self._safe_extract(card, 'h2.jobTitle a', 'href')
            if not link:
                link = self._safe_extract(card, 'a.jcs-JobTitle', 'href')
            if link and link.startswith('/'):
                link = 'https://ca.indeed.com' + link
            
            # Job Type (bonus field)
            job_type = self._extract_job_type(card)
            
            if title:
                return {
                    'title': title,
                    'company': company or "Company not found",
                    'location': location or "Location not specified",
                    'salary': salary,
                    'job_type': job_type,
                    'link': link or "Link not available",
                    'scraped_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
                }
                
        except Exception as e:
            print(f"Error extracting job info: {e}")
        
        return None

    def _safe_extract(self, parent, selector, attribute='text'):
        """Safely extract element text or attribute"""
        try:
            element = parent.find_element(By.CSS_SELECTOR, selector)
            if attribute == 'text':
                return element.text.strip()
            else:
                return element.get_attribute(attribute)
        except:
            return None

    def _extract_job_type(self, card):
        """Extract job type if available"""
        try:
            # Look for job type indicators in the card text
            card_text = card.text.lower()
            if 'full-time' in card_text:
                return 'Full-time'
            elif 'part-time' in card_text:
                return 'Part-time'
            elif 'contract' in card_text:
                return 'Contract'
            elif 'temporary' in card_text:
                return 'Temporary'
            elif 'remote' in card_text:
                return 'Remote'
            else:
                return 'Not specified'
        except:
            return 'Not specified'

    def save_to_csv(self, filename="indeed_canada_jobs_final.csv"):
        """Save results to CSV"""
        if not self.jobs_data:
            print("No jobs data to save.")
            return False
        
        df = pd.DataFrame(self.jobs_data)
        df.to_csv(filename, index=False, encoding='utf-8')
        print(f"Successfully saved {len(self.jobs_data)} jobs to {filename}")
        return True

    def display_sample(self, count=5):
        """Display sample of scraped jobs"""
        if not self.jobs_data:
            print("No jobs to display.")
            return
        
        print(f"\n{'='*80}")
        print(f"SAMPLE OF {min(count, len(self.jobs_data))} SCRAPED JOBS")
        print(f"{'='*80}")
        
        for i, job in enumerate(self.jobs_data[:count], 1):
            print(f"\n{i}. {job['title']}")
            print(f"   Company: {job['company']}")
            print(f"   Location: {job['location']}")
            print(f"   Salary: {job['salary']}")
            print(f"   Type: {job['job_type']}")
            print(f"   Link: {job['link'][:80]}...")

def main():
    print("🚀 Indeed Canada Job Scraper - Final Version")
    print("=" * 50)
    
    # Get user input
    job_title = input("Enter job title (default: developer): ").strip() or "developer"
    location = input("Enter location (default: Canada): ").strip() or "Canada"
    pages = input("Enter number of pages to scrape (default: 3): ").strip()
    pages = int(pages) if pages.isdigit() else 3
    
    print(f"\nScraping '{job_title}' jobs in '{location}' ({pages} pages)...")
    
    # Initialize and run scraper
    scraper = IndeedCanadaScraper(headless=True)
    
    start_time = time.time()
    jobs = scraper.scrape_jobs(
        job_title=job_title,
        location=location,
        max_pages=pages
    )
    end_time = time.time()
    
    # Display results
    if jobs:
        print(f"\n✅ Successfully scraped {len(jobs)} jobs in {end_time - start_time:.1f} seconds")
        
        # Save to CSV
        filename = f"indeed_{job_title.replace(' ', '_')}_{location.replace(' ', '_')}_jobs.csv"
        scraper.save_to_csv(filename)
        
        # Display sample
        scraper.display_sample(5)
        
        # Show statistics
        companies = set(job['company'] for job in jobs if job['company'] != "Company not found")
        locations = set(job['location'] for job in jobs if job['location'] != "Location not specified")
        remote_jobs = sum(1 for job in jobs if 'remote' in job['job_type'].lower())
        
        print(f"\n📊 STATISTICS:")
        print(f"   • Total jobs: {len(jobs)}")
        print(f"   • Unique companies: {len(companies)}")
        print(f"   • Locations: {len(locations)}")
        print(f"   • Remote jobs: {remote_jobs}")
        
    else:
        print("No jobs were scraped")

if __name__ == "__main__":
    main()

🚀 Indeed Canada Job Scraper - Final Version


Enter job title (default: developer):  developer
Enter location (default: Canada):  canada
Enter number of pages to scrape (default: 3):  5



Scraping 'developer' jobs in 'canada' (5 pages)...
Scraping page 1...
Found 16 job cards
Added 15 jobs from page 1
Scraping page 2...
No job cards found on page 2
Scraping page 3...
No job cards found on page 3
Scraping page 4...
No job cards found on page 4
Scraping page 5...
No job cards found on page 5

✅ Successfully scraped 15 jobs in 72.1 seconds
Successfully saved 15 jobs to indeed_developer_canada_jobs.csv

SAMPLE OF 5 SCRAPED JOBS

1. Wireless DSP Software Developer
   Company: Skycope C-UAS Technologies Inc
   Location: Burnaby, BC V5G 1J9
   Salary: $100,000–$150,000 a year
   Type: Full-time
   Link: https://ca.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0A7000hUPCMxGwW2vqsju70Hf0EmrA...

2. Intermediate Full Stack Software Engineer
   Company: D3 Security Management Systems
   Location: Vancouver, BC V6C 1G8
   Salary: $70,000–$100,000 a year
   Type: Full-time
   Link: https://ca.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0BvCuWiBIaNCSUz0Q4GXryJc0qOCu-...

3. CNC Programmer/Opera

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import random
import re
import jaydebeapi
import os
from datetime import datetime

class IndeedScraperWithH2:
    def __init__(self, headless=True):
        self.options = Options()
        
        self.options.add_argument('--no-sandbox')
        self.options.add_argument('--disable-dev-shm-usage')
        self.options.add_argument('--window-size=1920,1080')
        self.options.add_argument('--disable-blink-features=AutomationControlled')
        
        self.options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
        
        if headless:
            self.options.add_argument('--headless')
        
        self.driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()),
            options=self.options
        )
        
        self.jobs_data = []
        self.db_connection = None
        
    def setup_h2_database(self, db_path="indeed_jobs.db"):
        """Setup H2 database connection"""
        try:
            # H2 JDBC driver class and connection string
            jdbc_driver = 'org.h2.Driver'
            jdbc_url = f'jdbc:h2:file:./{db_path}'
            
            # Connect to H2 database (it will create if doesn't exist)
            self.db_connection = jaydebeapi.connect(
                jdbc_driver,
                jdbc_url,
                ['sa', '']  # username, password
            )
            
            # Create jobs table if it doesn't exist
            self._create_jobs_table()
            print(f"✅ H2 database connected: {db_path}")
            return True
            
        except Exception as e:
            print(f"❌ Error setting up H2 database: {e}")
            return False

    def _create_jobs_table(self):
        """Create the jobs table if it doesn't exist"""
        create_table_sql = """
        CREATE TABLE IF NOT EXISTS jobs (
            id BIGINT AUTO_INCREMENT PRIMARY KEY,
            title VARCHAR(500) NOT NULL,
            company VARCHAR(255),
            location VARCHAR(255),
            salary VARCHAR(100),
            job_type VARCHAR(50),
            link VARCHAR(1000),
            scraped_date TIMESTAMP,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            UNIQUE(title, company, location)
        )
        """
        
        cursor = self.db_connection.cursor()
        cursor.execute(create_table_sql)
        cursor.close()
        print("✅ Jobs table created/verified")

    def scrape_jobs(self, job_title="developer", location="Canada", max_pages=3):
        """Scrape jobs and automatically save to H2 database"""
        
        try:
            for page in range(max_pages):
                if page == 0:
                    url = f"https://ca.indeed.com/jobs?q={job_title.replace(' ', '+')}&l={location.replace(' ', '+').replace(',', '%2C')}"
                else:
                    url = f"https://ca.indeed.com/jobs?q={job_title.replace(' ', '+')}&l={location.replace(' ', '+').replace(',', '%2C')}&start={page * 10}"
                
                print(f"Scraping page {page + 1}...")
                
                self.driver.get(url)
                time.sleep(random.uniform(3, 5))
                
                # Wait for job cards
                try:
                    WebDriverWait(self.driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.job_seen_beacon'))
                    )
                except:
                    print(f"No job cards found on page {page + 1}")
                    continue
                
                jobs_count_before = len(self.jobs_data)
                self._extract_page_jobs_css()
                jobs_added = len(self.jobs_data) - jobs_count_before
                
                print(f"Added {jobs_added} jobs from page {page + 1}")
                
                # Save to database after each page
                if jobs_added > 0:
                    saved_count = self.save_to_database()
                    print(f"💾 Saved {saved_count} jobs to database")
                
                if page < max_pages - 1:
                    time.sleep(random.uniform(2, 4))
                    
        except Exception as e:
            print(f"Error during scraping: {e}")
        
        finally:
            self.driver.quit()
            if self.db_connection:
                self.db_connection.close()
                print("✅ Database connection closed")
        
        return self.jobs_data

    def _extract_page_jobs_css(self):
        """Extract jobs using reliable CSS selectors"""
        try:
            job_cards = self.driver.find_elements(By.CSS_SELECTOR, 'div.job_seen_beacon')
            print(f"Found {len(job_cards)} job cards")
            
            for card in job_cards:
                job_info = self._extract_job_info_css(card)
                if job_info:
                    self.jobs_data.append(job_info)
                    
        except Exception as e:
            print(f"Error extracting jobs: {e}")

    def _extract_job_info_css(self, card):
        """Extract job information using CSS selectors"""
        try:
            # Title
            title = self._safe_extract(card, 'h2.jobTitle', 'text')
            if title:
                title = re.sub(r'^New\s*', '', title).strip()
            
            # Company
            company = self._safe_extract(card, '[data-testid="company-name"]', 'text')
            if not company:
                company = self._safe_extract(card, '[class*="companyName"]', 'text')
            
            # Location
            location = self._safe_extract(card, '[data-testid="text-location"]', 'text')
            if not location:
                location = self._safe_extract(card, '[class*="companyLocation"]', 'text')
            
            # Salary
            salary = "Salary not specified"
            salary_text = self._safe_extract(card, '[data-testid="attribute_snippet_testid"]', 'text')
            if salary_text and '$' in salary_text:
                salary = salary_text
            
            # Link
            link = self._safe_extract(card, 'h2.jobTitle a', 'href')
            if link and link.startswith('/'):
                link = 'https://ca.indeed.com' + link
            
            # Job Type
            job_type = self._extract_job_type(card)
            
            if title:
                return {
                    'title': title,
                    'company': company or "Company not found",
                    'location': location or "Location not specified",
                    'salary': salary,
                    'job_type': job_type,
                    'link': link or "Link not available",
                    'scraped_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                }
                
        except Exception as e:
            print(f"Error extracting job info: {e}")
        
        return None

    def _safe_extract(self, parent, selector, attribute='text'):
        """Safely extract element text or attribute"""
        try:
            element = parent.find_element(By.CSS_SELECTOR, selector)
            if attribute == 'text':
                return element.text.strip()
            else:
                return element.get_attribute(attribute)
        except:
            return None

    def _extract_job_type(self, card):
        """Extract job type"""
        try:
            card_text = card.text.lower()
            if 'full-time' in card_text:
                return 'Full-time'
            elif 'part-time' in card_text:
                return 'Part-time'
            elif 'contract' in card_text:
                return 'Contract'
            elif 'temporary' in card_text:
                return 'Temporary'
            elif 'remote' in card_text:
                return 'Remote'
            else:
                return 'Not specified'
        except:
            return 'Not specified'

    def save_to_database(self):
        """Save current jobs data to H2 database"""
        if not self.jobs_data or not self.db_connection:
            print("No data to save or database not connected")
            return 0
        
        try:
            cursor = self.db_connection.cursor()
            saved_count = 0
            
            insert_sql = """
            INSERT INTO jobs (title, company, location, salary, job_type, link, scraped_date)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            """
            
            for job in self.jobs_data:
                try:
                    cursor.execute(insert_sql, (
                        job['title'],
                        job['company'],
                        job['location'],
                        job['salary'],
                        job['job_type'],
                        job['link'],
                        job['scraped_date']
                    ))
                    saved_count += 1
                except Exception as e:
                    # This might be a duplicate, which is fine due to UNIQUE constraint
                    continue
            
            self.db_connection.commit()
            cursor.close()
            return saved_count
            
        except Exception as e:
            print(f"Error saving to database: {e}")
            return 0

    def query_jobs_from_database(self, limit=10):
        """Query jobs from H2 database"""
        if not self.db_connection:
            print("Database not connected")
            return []
        
        try:
            cursor = self.db_connection.cursor()
            cursor.execute("""
                SELECT title, company, location, salary, job_type, link, scraped_date 
                FROM jobs 
                ORDER BY scraped_date DESC 
                LIMIT ?
            """, (limit,))
            
            results = cursor.fetchall()
            cursor.close()
            
            jobs = []
            for row in results:
                jobs.append({
                    'title': row[0],
                    'company': row[1],
                    'location': row[2],
                    'salary': row[3],
                    'job_type': row[4],
                    'link': row[5],
                    'scraped_date': row[6]
                })
            
            return jobs
            
        except Exception as e:
            print(f"Error querying database: {e}")
            return []

    def get_database_stats(self):
        """Get statistics from the database"""
        if not self.db_connection:
            print("Database not connected")
            return {}
        
        try:
            cursor = self.db_connection.cursor()
            
            # Total jobs count
            cursor.execute("SELECT COUNT(*) FROM jobs")
            total_jobs = cursor.fetchone()[0]
            
            # Companies count
            cursor.execute("SELECT COUNT(DISTINCT company) FROM jobs")
            unique_companies = cursor.fetchone()[0]
            
            # Locations count
            cursor.execute("SELECT COUNT(DISTINCT location) FROM jobs")
            unique_locations = cursor.fetchone()[0]
            
            # Latest scrape date
            cursor.execute("SELECT MAX(scraped_date) FROM jobs")
            latest_scrape = cursor.fetchone()[0]
            
            cursor.close()
            
            return {
                'total_jobs': total_jobs,
                'unique_companies': unique_companies,
                'unique_locations': unique_locations,
                'latest_scrape': latest_scrape
            }
            
        except Exception as e:
            print(f"Error getting database stats: {e}")
            return {}

    def save_to_csv(self, filename="indeed_jobs_backup.csv"):
        """Backup database to CSV file"""
        if not self.db_connection:
            print("Database not connected")
            return False
        
        try:
            cursor = self.db_connection.cursor()
            cursor.execute("SELECT * FROM jobs")
            
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
            
            df = pd.DataFrame(data, columns=columns)
            df.to_csv(filename, index=False)
            
            cursor.close()
            print(f"💾 Database backed up to {filename}")
            return True
            
        except Exception as e:
            print(f"Error backing up to CSV: {e}")
            return False

def main():
    print("🚀 Indeed Scraper with H2 Database")
    print("=" * 50)
    
    # Initialize scraper with H2 database
    scraper = IndeedScraperWithH2(headless=True)
    
    # Setup H2 database
    if not scraper.setup_h2_database("indeed_jobs_db"):
        print("Failed to setup database. Exiting.")
        return
    
    # Get user input
    job_title = input("Enter job title (default: developer): ").strip() or "developer"
    location = input("Enter location (default: Canada): ").strip() or "Canada"
    pages = input("Enter number of pages to scrape (default: 2): ").strip()
    pages = int(pages) if pages.isdigit() else 2
    
    print(f"\nScraping '{job_title}' jobs in '{location}' ({pages} pages)...")
    
    # Scrape and save to database
    start_time = time.time()
    jobs = scraper.scrape_jobs(
        job_title=job_title,
        location=location,
        max_pages=pages
    )
    end_time = time.time()
    
    # Display results
    if jobs:
        print(f"\n✅ Successfully processed {len(jobs)} jobs in {end_time - start_time:.1f} seconds")
        
        # Get database statistics
        stats = scraper.get_database_stats()
        print(f"\n📊 DATABASE STATISTICS:")
        print(f"   • Total jobs in database: {stats.get('total_jobs', 0)}")
        print(f"   • Unique companies: {stats.get('unique_companies', 0)}")
        print(f"   • Unique locations: {stats.get('unique_locations', 0)}")
        print(f"   • Latest scrape: {stats.get('latest_scrape', 'N/A')}")
        
        # Query and display recent jobs from database
        recent_jobs = scraper.query_jobs_from_database(limit=5)
        if recent_jobs:
            print(f"\n📋 RECENT JOBS FROM DATABASE:")
            for i, job in enumerate(recent_jobs, 1):
                print(f"\n{i}. {job['title']}")
                print(f"   Company: {job['company']}")
                print(f"   Location: {job['location']}")
                print(f"   Salary: {job['salary']}")
                print(f"   Scraped: {job['scraped_date']}")
        
        # Backup to CSV
        scraper.save_to_csv("indeed_jobs_backup.csv")
        
    else:
        print("❌ No jobs were scraped")
    
    print(f"\n💾 Database file: indeed_jobs_db.mv.db")
    print("🔧 You can query the database using any SQL client that supports H2")

if __name__ == "__main__":
    main()

🚀 Indeed Scraper with H2 Database
❌ Error setting up H2 database: Class org.h2.Driver is not found
Failed to setup database. Exiting.


In [16]:
pip install jaydebeapi

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import requests
import subprocess
import sys
from pathlib import Path

def setup_h2_driver():
    """Download H2 database driver if not present"""
    h2_jar_path = "h2-2.2.224.jar"
    h2_url = "https://repo1.maven.org/maven2/com/h2database/h2/2.2.224/h2-2.2.224.jar"
    
    if not os.path.exists(h2_jar_path):
        print("📥 Downloading H2 database driver...")
        try:
            response = requests.get(h2_url, stream=True)
            response.raise_for_status()
            
            with open(h2_jar_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print("✅ H2 driver downloaded successfully")
        except Exception as e:
            print(f"❌ Error downloading H2 driver: {e}")
            return False
    
    return True

# Install required packages
def install_packages():
    """Install required Python packages"""
    packages = ['jaydebeapi', 'JPype1', 'selenium', 'pandas', 'webdriver-manager', 'requests']
    
    for package in packages:
        try:
            __import__(package.replace('-', '_'))
            print(f"✅ {package} already installed")
        except ImportError:
            print(f"📦 Installing {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Run setup
if __name__ == "__main__":
    print("🚀 Setting up H2 database environment...")
    install_packages()
    setup_h2_driver()
    print("✅ Setup complete! Now run the main scraper.")

🚀 Setting up H2 database environment...
✅ jaydebeapi already installed
📦 Installing JPype1...
✅ selenium already installed
✅ pandas already installed
✅ webdriver-manager already installed
✅ requests already installed
📥 Downloading H2 database driver...
✅ H2 driver downloaded successfully
✅ Setup complete! Now run the main scraper.


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import jaydebeapi
import pandas as pd
import time
import random
import re
import os
from datetime import datetime

class IndeedScraperWithH2:
    def __init__(self, headless=True):
        self.options = Options()
        
        self.options.add_argument('--no-sandbox')
        self.options.add_argument('--disable-dev-shm-usage')
        self.options.add_argument('--window-size=1920,1080')
        self.options.add_argument('--disable-blink-features=AutomationControlled')
        
        self.options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
        
        if headless:
            self.options.add_argument('--headless')
        
        self.driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()),
            options=self.options
        )
        
        self.jobs_data = []
        self.db_connection = None
        self.h2_jar_path = "h2-2.2.224.jar"
        
    def setup_h2_database(self, db_path="./indeed_jobs"):
        """Setup H2 database connection with proper JAR path"""
        try:
            if not os.path.exists(self.h2_jar_path):
                print("❌ H2 driver JAR not found. Please run setup script first.")
                return False
            
            # H2 JDBC connection details
            jdbc_driver = "org.h2.Driver"
            jdbc_url = f"jdbc:h2:file:{db_path}"
            
            print(f"🔗 Connecting to H2 database: {db_path}")
            
            # Connect to H2 database
            self.db_connection = jaydebeapi.connect(
                jdbc_driver,
                jdbc_url,
                ["sa", ""],  # username, password
                self.h2_jar_path
            )
            
            # Create jobs table
            self._create_jobs_table()
            print("✅ H2 database connected and ready")
            return True
            
        except Exception as e:
            print(f"❌ Error setting up H2 database: {e}")
            return False

    def _create_jobs_table(self):
        """Create the jobs table in H2 database"""
        create_table_sql = """
        CREATE TABLE IF NOT EXISTS jobs (
            id BIGINT AUTO_INCREMENT PRIMARY KEY,
            title VARCHAR(500) NOT NULL,
            company VARCHAR(255),
            location VARCHAR(255),
            salary VARCHAR(100),
            job_type VARCHAR(50),
            link VARCHAR(1000),
            scraped_date TIMESTAMP,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            UNIQUE(title, company, location)
        )
        """
        
        cursor = self.db_connection.cursor()
        cursor.execute(create_table_sql)
        cursor.close()
        self.db_connection.commit()
        print("✅ Jobs table created/verified in H2 database")

    def scrape_jobs(self, job_title="developer", location="Canada", max_pages=3):
        """Scrape jobs and save to H2 database"""
        
        try:
            for page in range(max_pages):
                if page == 0:
                    url = f"https://ca.indeed.com/jobs?q={job_title.replace(' ', '+')}&l={location.replace(' ', '+').replace(',', '%2C')}"
                else:
                    url = f"https://ca.indeed.com/jobs?q={job_title.replace(' ', '+')}&l={location.replace(' ', '+').replace(',', '%2C')}&start={page * 10}"
                
                print(f"🌐 Scraping page {page + 1}...")
                
                self.driver.get(url)
                time.sleep(random.uniform(3, 5))
                
                # Wait for job cards
                try:
                    WebDriverWait(self.driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.job_seen_beacon'))
                    )
                except:
                    print(f"❌ No job cards found on page {page + 1}")
                    continue
                
                jobs_count_before = len(self.jobs_data)
                self._extract_page_jobs_css()
                jobs_added = len(self.jobs_data) - jobs_count_before
                
                print(f"✅ Added {jobs_added} jobs from page {page + 1}")
                
                # Save to H2 database after each page
                if jobs_added > 0 and self.db_connection:
                    saved_count = self.save_to_h2_database()
                    print(f"💾 Saved {saved_count} jobs to H2 database")
                
                if page < max_pages - 1:
                    time.sleep(random.uniform(2, 4))
                    
        except Exception as e:
            print(f"❌ Error during scraping: {e}")
        
        finally:
            self.driver.quit()
            if self.db_connection:
                self.db_connection.close()
                print("✅ H2 database connection closed")
        
        return self.jobs_data

    def _extract_page_jobs_css(self):
        """Extract jobs from current page"""
        try:
            job_cards = self.driver.find_elements(By.CSS_SELECTOR, 'div.job_seen_beacon')
            print(f"📄 Found {len(job_cards)} job cards")
            
            for card in job_cards:
                job_info = self._extract_job_info_css(card)
                if job_info:
                    self.jobs_data.append(job_info)
                    
        except Exception as e:
            print(f"❌ Error extracting jobs: {e}")

    def _extract_job_info_css(self, card):
        """Extract job information from card"""
        try:
            # Title
            title = self._safe_extract(card, 'h2.jobTitle', 'text')
            if title:
                title = re.sub(r'^New\s*', '', title).strip()
            
            # Company
            company = self._safe_extract(card, '[data-testid="company-name"]', 'text')
            if not company:
                company = self._safe_extract(card, '[class*="companyName"]', 'text')
            
            # Location
            location = self._safe_extract(card, '[data-testid="text-location"]', 'text')
            if not location:
                location = self._safe_extract(card, '[class*="companyLocation"]', 'text')
            
            # Salary
            salary = "Salary not specified"
            salary_text = self._safe_extract(card, '[data-testid="attribute_snippet_testid"]', 'text')
            if salary_text and '$' in salary_text:
                salary = salary_text
            
            # Link
            link = self._safe_extract(card, 'h2.jobTitle a', 'href')
            if link and link.startswith('/'):
                link = 'https://ca.indeed.com' + link
            
            # Job Type
            job_type = self._extract_job_type(card)
            
            if title:
                return {
                    'title': title,
                    'company': company or "Company not found",
                    'location': location or "Location not specified",
                    'salary': salary,
                    'job_type': job_type,
                    'link': link or "Link not available",
                    'scraped_date': datetime.now()
                }
                
        except Exception as e:
            print(f"❌ Error extracting job info: {e}")
        
        return None

    def _safe_extract(self, parent, selector, attribute='text'):
        """Safely extract element text or attribute"""
        try:
            element = parent.find_element(By.CSS_SELECTOR, selector)
            if attribute == 'text':
                return element.text.strip()
            else:
                return element.get_attribute(attribute)
        except:
            return None

    def _extract_job_type(self, card):
        """Extract job type from card"""
        try:
            card_text = card.text.lower()
            if 'full-time' in card_text:
                return 'Full-time'
            elif 'part-time' in card_text:
                return 'Part-time'
            elif 'contract' in card_text:
                return 'Contract'
            elif 'temporary' in card_text:
                return 'Temporary'
            elif 'remote' in card_text:
                return 'Remote'
            else:
                return 'Not specified'
        except:
            return 'Not specified'

    def save_to_h2_database(self):
        """Save jobs data to H2 database"""
        if not self.jobs_data or not self.db_connection:
            print("❌ No data to save or database not connected")
            return 0
        
        try:
            cursor = self.db_connection.cursor()
            saved_count = 0
            
            insert_sql = """
            MERGE INTO jobs (title, company, location, salary, job_type, link, scraped_date)
            KEY(title, company, location)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            """
            
            for job in self.jobs_data:
                try:
                    cursor.execute(insert_sql, (
                        job['title'],
                        job['company'],
                        job['location'],
                        job['salary'],
                        job['job_type'],
                        job['link'],
                        job['scraped_date']
                    ))
                    saved_count += 1
                except Exception as e:
                    # Duplicate entry, skip
                    continue
            
            self.db_connection.commit()
            cursor.close()
            return saved_count
            
        except Exception as e:
            print(f"❌ Error saving to H2 database: {e}")
            return 0

    def query_h2_database(self, limit=10):
        """Query jobs from H2 database"""
        if not self.db_connection:
            print("❌ Database not connected")
            return []
        
        try:
            cursor = self.db_connection.cursor()
            cursor.execute("""
                SELECT title, company, location, salary, job_type, link, scraped_date 
                FROM jobs 
                ORDER BY scraped_date DESC 
                LIMIT ?
            """, (limit,))
            
            results = cursor.fetchall()
            cursor.close()
            
            jobs = []
            for row in results:
                jobs.append({
                    'title': row[0],
                    'company': row[1],
                    'location': row[2],
                    'salary': row[3],
                    'job_type': row[4],
                    'link': row[5],
                    'scraped_date': row[6]
                })
            
            return jobs
            
        except Exception as e:
            print(f"❌ Error querying H2 database: {e}")
            return []

    def get_h2_database_stats(self):
        """Get statistics from H2 database"""
        if not self.db_connection:
            print("❌ Database not connected")
            return {}
        
        try:
            cursor = self.db_connection.cursor()
            
            # Total jobs count
            cursor.execute("SELECT COUNT(*) FROM jobs")
            total_jobs = cursor.fetchone()[0]
            
            # Companies count
            cursor.execute("SELECT COUNT(DISTINCT company) FROM jobs")
            unique_companies = cursor.fetchone()[0]
            
            # Locations count
            cursor.execute("SELECT COUNT(DISTINCT location) FROM jobs")
            unique_locations = cursor.fetchone()[0]
            
            # Latest scrape date
            cursor.execute("SELECT MAX(scraped_date) FROM jobs")
            latest_scrape = cursor.fetchone()[0]
            
            cursor.close()
            
            return {
                'total_jobs': total_jobs,
                'unique_companies': unique_companies,
                'unique_locations': unique_locations,
                'latest_scrape': latest_scrape
            }
            
        except Exception as e:
            print(f"❌ Error getting H2 database stats: {e}")
            return {}

    def export_h2_to_csv(self, filename="indeed_jobs_h2_backup.csv"):
        """Export H2 database to CSV"""
        if not self.db_connection:
            print("❌ Database not connected")
            return False
        
        try:
            cursor = self.db_connection.cursor()
            cursor.execute("SELECT * FROM jobs")
            
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
            
            df = pd.DataFrame(data, columns=columns)
            df.to_csv(filename, index=False)
            
            cursor.close()
            print(f"💾 H2 database backed up to {filename}")
            return True
            
        except Exception as e:
            print(f"❌ Error backing up H2 to CSV: {e}")
            return False

def main():
    print("🚀 Indeed Scraper with H2 Database")
    print("=" * 50)
    
    # Check if H2 driver exists
    if not os.path.exists("h2-2.2.224.jar"):
        print("❌ H2 driver not found. Please run the setup script first.")
        print("💡 Run: python setup_h2.py")
        return
    
    # Initialize scraper
    scraper = IndeedScraperWithH2(headless=True)
    
    # Setup H2 database
    if not scraper.setup_h2_database("./indeed_jobs_db"):
        print("❌ Failed to setup H2 database. Exiting.")
        return
    
    # Get user input
    job_title = input("Enter job title (default: developer): ").strip() or "developer"
    location = input("Enter location (default: Canada): ").strip() or "Canada"
    pages = input("Enter number of pages to scrape (default: 2): ").strip()
    pages = int(pages) if pages.isdigit() else 2
    
    print(f"\n🎯 Scraping '{job_title}' jobs in '{location}' ({pages} pages)...")
    
    # Scrape and save to H2 database
    start_time = time.time()
    jobs = scraper.scrape_jobs(
        job_title=job_title,
        location=location,
        max_pages=pages
    )
    end_time = time.time()
    
    # Display results
    if jobs:
        print(f"\n✅ Successfully processed {len(jobs)} jobs in {end_time - start_time:.1f} seconds")
        
        # Get H2 database statistics
        stats = scraper.get_h2_database_stats()
        print(f"\n📊 H2 DATABASE STATISTICS:")
        print(f"   • Total jobs in H2: {stats.get('total_jobs', 0)}")
        print(f"   • Unique companies: {stats.get('unique_companies', 0)}")
        print(f"   • Unique locations: {stats.get('unique_locations', 0)}")
        print(f"   • Latest scrape: {stats.get('latest_scrape', 'N/A')}")
        
        # Query and display recent jobs from H2 database
        recent_jobs = scraper.query_h2_database(limit=5)
        if recent_jobs:
            print(f"\n📋 RECENT JOBS FROM H2 DATABASE:")
            for i, job in enumerate(recent_jobs, 1):
                print(f"\n{i}. {job['title']}")
                print(f"   Company: {job['company']}")
                print(f"   Location: {job['location']}")
                print(f"   Salary: {job['salary']}")
                print(f"   Scraped: {job['scraped_date']}")
        
        # Backup H2 to CSV
        scraper.export_h2_to_csv("indeed_jobs_h2_backup.csv")
        
    else:
        print("❌ No jobs were scraped")
    
    print(f"\n💾 H2 database files:")
    print(f"   - indeed_jobs_db.mv.db (main database file)")
    print(f"   - indeed_jobs_db.trace.db (log file)")
    print(f"   - indeed_jobs_h2_backup.csv (CSV backup)")
    
    print(f"\n🔧 You can connect to H2 database using:")
    print(f"   JDBC URL: jdbc:h2:file:./indeed_jobs_db")
    print(f"   Username: sa")
    print(f"   Password: (empty)")

if __name__ == "__main__":
    main()

🚀 Indeed Scraper with H2 Database
🔗 Connecting to H2 database: ./indeed_jobs_db
❌ Error setting up H2 database: Class org.h2.Driver is not found
❌ Failed to setup H2 database. Exiting.


In [5]:
# setup_h2.py
import os
import requests
import subprocess
import sys

def download_h2_jar():
    """Download H2 database JAR file"""
    h2_version = "2.2.224"
    h2_jar_name = f"h2-{h2_version}.jar"
    h2_url = f"https://repo1.maven.org/maven2/com/h2database/h2/{h2_version}/{h2_jar_name}"
    
    if not os.path.exists(h2_jar_name):
        print(f"📥 Downloading H2 database {h2_version}...")
        try:
            response = requests.get(h2_url, stream=True)
            response.raise_for_status()
            
            with open(h2_jar_name, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f"✅ H2 driver downloaded: {h2_jar_name}")
            return h2_jar_name
        except Exception as e:
            print(f"❌ Error downloading H2: {e}")
            return None
    else:
        print(f"✅ H2 driver already exists: {h2_jar_name}")
        return h2_jar_name

def install_requirements():
    """Install required Python packages"""
    requirements = [
        'jaydebeapi',
        'JPype1==1.4.1',
        'selenium',
        'pandas',
        'webdriver-manager',
        'requests'
    ]
    
    for package in requirements:
        try:
            if '==' in package:
                package_name = package.split('==')[0]
            else:
                package_name = package
                
            __import__(package_name.replace('-', '_'))
            print(f"✅ {package} already installed")
        except ImportError:
            print(f"📦 Installing {package}...")
            try:
                subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            except subprocess.CalledProcessError as e:
                print(f"⚠️  Failed to install {package}: {e}")

if __name__ == "__main__":
    print("🚀 Setting up H2 Database Environment...")
    install_requirements()
    jar_file = download_h2_jar()
    
    if jar_file:
        print(f"\n✅ Setup complete!")
        print(f"📁 H2 JAR file: {jar_file}")
        print(f"🔧 Now run: python indeed_scraper_h2.py")
    else:
        print("❌ Setup failed!")

🚀 Setting up H2 Database Environment...
✅ jaydebeapi already installed
📦 Installing JPype1==1.4.1...
⚠️  Failed to install JPype1==1.4.1: Command '['C:\\Users\\lesha\\anaconda3\\python.exe', '-m', 'pip', 'install', 'JPype1==1.4.1']' returned non-zero exit status 1.
✅ selenium already installed
✅ pandas already installed
✅ webdriver-manager already installed
✅ requests already installed
✅ H2 driver already exists: h2-2.2.224.jar

✅ Setup complete!
📁 H2 JAR file: h2-2.2.224.jar
🔧 Now run: python indeed_scraper_h2.py


In [6]:
# indeed_scraper_h2.py
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import jaydebeapi
import pandas as pd
import time
import random
import re
import os
from datetime import datetime

class IndeedScraperWithH2:
    def __init__(self, headless=True):
        self.options = Options()
        
        self.options.add_argument('--no-sandbox')
        self.options.add_argument('--disable-dev-shm-usage')
        self.options.add_argument('--window-size=1920,1080')
        self.options.add_argument('--disable-blink-features=AutomationControlled')
        
        self.options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
        
        if headless:
            self.options.add_argument('--headless')
        
        self.driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()),
            options=self.options
        )
        
        self.jobs_data = []
        self.db_connection = None
        
    def setup_h2_database(self, db_name="indeed_jobs"):
        """Setup H2 database connection with proper JAR loading"""
        try:
            # Find H2 JAR file
            h2_jar = self._find_h2_jar()
            if not h2_jar:
                print("❌ H2 JAR file not found. Please run setup_h2.py first.")
                return False
            
            print(f"🔧 Using H2 JAR: {h2_jar}")
            
            # H2 JDBC connection details
            jdbc_driver = "org.h2.Driver"
            jdbc_url = f"jdbc:h2:file:./{db_name};DB_CLOSE_DELAY=-1"
            
            print(f"🔗 Connecting to H2 database: {jdbc_url}")
            
            # Connect to H2 database with explicit JAR path
            self.db_connection = jaydebeapi.connect(
                jdbc_driver,
                jdbc_url,
                ["sa", ""],  # username, password
                h2_jar
            )
            
            # Create jobs table
            self._create_jobs_table()
            print("✅ H2 database connected and ready")
            return True
            
        except Exception as e:
            print(f"❌ Error setting up H2 database: {e}")
            import traceback
            traceback.print_exc()
            return False

    def _find_h2_jar(self):
        """Find the H2 JAR file in current directory"""
        for file in os.listdir('.'):
            if file.startswith('h2-') and file.endswith('.jar'):
                return file
        return None

    def _create_jobs_table(self):
        """Create the jobs table in H2 database"""
        create_table_sql = """
        CREATE TABLE IF NOT EXISTS jobs (
            id BIGINT AUTO_INCREMENT PRIMARY KEY,
            title VARCHAR(500) NOT NULL,
            company VARCHAR(255),
            location VARCHAR(255),
            salary VARCHAR(100),
            job_type VARCHAR(50),
            link VARCHAR(1000),
            scraped_date TIMESTAMP,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        """
        
        cursor = self.db_connection.cursor()
        cursor.execute(create_table_sql)
        cursor.close()
        self.db_connection.commit()
        print("✅ Jobs table created/verified in H2 database")

    def scrape_jobs(self, job_title="developer", location="Canada", max_pages=3):
        """Scrape jobs and save to H2 database"""
        
        try:
            for page in range(max_pages):
                if page == 0:
                    url = f"https://ca.indeed.com/jobs?q={job_title.replace(' ', '+')}&l={location.replace(' ', '+').replace(',', '%2C')}"
                else:
                    url = f"https://ca.indeed.com/jobs?q={job_title.replace(' ', '+')}&l={location.replace(' ', '+').replace(',', '%2C')}&start={page * 10}"
                
                print(f"🌐 Scraping page {page + 1}...")
                
                self.driver.get(url)
                time.sleep(random.uniform(3, 5))
                
                # Wait for job cards
                try:
                    WebDriverWait(self.driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.job_seen_beacon'))
                    )
                except:
                    print(f"❌ No job cards found on page {page + 1}")
                    continue
                
                jobs_count_before = len(self.jobs_data)
                self._extract_page_jobs_css()
                jobs_added = len(self.jobs_data) - jobs_count_before
                
                print(f"✅ Added {jobs_added} jobs from page {page + 1}")
                
                # Save to H2 database after each page
                if jobs_added > 0 and self.db_connection:
                    saved_count = self.save_to_h2_database()
                    print(f"💾 Saved {saved_count} jobs to H2 database")
                
                if page < max_pages - 1:
                    time.sleep(random.uniform(2, 4))
                    
        except Exception as e:
            print(f"❌ Error during scraping: {e}")
            import traceback
            traceback.print_exc()
        
        finally:
            self.driver.quit()
            if self.db_connection:
                self.db_connection.close()
                print("✅ H2 database connection closed")
        
        return self.jobs_data

    def _extract_page_jobs_css(self):
        """Extract jobs from current page"""
        try:
            job_cards = self.driver.find_elements(By.CSS_SELECTOR, 'div.job_seen_beacon')
            print(f"📄 Found {len(job_cards)} job cards")
            
            for card in job_cards:
                job_info = self._extract_job_info_css(card)
                if job_info:
                    self.jobs_data.append(job_info)
                    
        except Exception as e:
            print(f"❌ Error extracting jobs: {e}")

    def _extract_job_info_css(self, card):
        """Extract job information from card"""
        try:
            # Title
            title = self._safe_extract(card, 'h2.jobTitle', 'text')
            if title:
                title = re.sub(r'^New\s*', '', title).strip()
            
            # Company
            company = self._safe_extract(card, '[data-testid="company-name"]', 'text')
            if not company:
                company = self._safe_extract(card, '[class*="companyName"]', 'text')
            
            # Location
            location = self._safe_extract(card, '[data-testid="text-location"]', 'text')
            if not location:
                location = self._safe_extract(card, '[class*="companyLocation"]', 'text')
            
            # Salary
            salary = "Salary not specified"
            salary_text = self._safe_extract(card, '[data-testid="attribute_snippet_testid"]', 'text')
            if salary_text and '$' in salary_text:
                salary = salary_text
            
            # Link
            link = self._safe_extract(card, 'h2.jobTitle a', 'href')
            if link and link.startswith('/'):
                link = 'https://ca.indeed.com' + link
            
            # Job Type
            job_type = self._extract_job_type(card)
            
            if title:
                return {
                    'title': title,
                    'company': company or "Company not found",
                    'location': location or "Location not specified",
                    'salary': salary,
                    'job_type': job_type,
                    'link': link or "Link not available",
                    'scraped_date': datetime.now()
                }
                
        except Exception as e:
            print(f"❌ Error extracting job info: {e}")
        
        return None

    def _safe_extract(self, parent, selector, attribute='text'):
        """Safely extract element text or attribute"""
        try:
            element = parent.find_element(By.CSS_SELECTOR, selector)
            if attribute == 'text':
                return element.text.strip()
            else:
                return element.get_attribute(attribute)
        except:
            return None

    def _extract_job_type(self, card):
        """Extract job type from card"""
        try:
            card_text = card.text.lower()
            if 'full-time' in card_text:
                return 'Full-time'
            elif 'part-time' in card_text:
                return 'Part-time'
            elif 'contract' in card_text:
                return 'Contract'
            elif 'temporary' in card_text:
                return 'Temporary'
            elif 'remote' in card_text:
                return 'Remote'
            else:
                return 'Not specified'
        except:
            return 'Not specified'

    def save_to_h2_database(self):
        """Save jobs data to H2 database"""
        if not self.jobs_data or not self.db_connection:
            print("❌ No data to save or database not connected")
            return 0
        
        try:
            cursor = self.db_connection.cursor()
            saved_count = 0
            
            # Use INSERT IGNORE to handle duplicates
            insert_sql = """
            INSERT INTO jobs (title, company, location, salary, job_type, link, scraped_date)
            SELECT ?, ?, ?, ?, ?, ?, ?
            WHERE NOT EXISTS (
                SELECT 1 FROM jobs 
                WHERE title = ? AND company = ? AND location = ?
            )
            """
            
            for job in self.jobs_data:
                try:
                    cursor.execute(insert_sql, (
                        job['title'], job['company'], job['location'], job['salary'], 
                        job['job_type'], job['link'], job['scraped_date'],
                        job['title'], job['company'], job['location']
                    ))
                    if cursor.rowcount > 0:
                        saved_count += 1
                except Exception as e:
                    print(f"⚠️  Error inserting job: {e}")
                    continue
            
            self.db_connection.commit()
            cursor.close()
            return saved_count
            
        except Exception as e:
            print(f"❌ Error saving to H2 database: {e}")
            import traceback
            traceback.print_exc()
            return 0

    def get_h2_database_stats(self):
        """Get statistics from H2 database"""
        if not self.db_connection:
            # Try to reconnect to get stats
            if not self.setup_h2_database():
                return {}
        
        try:
            cursor = self.db_connection.cursor()
            
            # Total jobs count
            cursor.execute("SELECT COUNT(*) FROM jobs")
            total_jobs = cursor.fetchone()[0]
            
            # Companies count
            cursor.execute("SELECT COUNT(DISTINCT company) FROM jobs")
            unique_companies = cursor.fetchone()[0]
            
            # Locations count
            cursor.execute("SELECT COUNT(DISTINCT location) FROM jobs")
            unique_locations = cursor.fetchone()[0]
            
            cursor.close()
            
            return {
                'total_jobs': total_jobs,
                'unique_companies': unique_companies,
                'unique_locations': unique_locations
            }
            
        except Exception as e:
            print(f"❌ Error getting H2 database stats: {e}")
            return {}

def main():
    print("🚀 Indeed Scraper with H2 Database")
    print("=" * 50)
    
    # Initialize scraper
    scraper = IndeedScraperWithH2(headless=True)
    
    # Setup H2 database
    if not scraper.setup_h2_database("indeed_jobs_db"):
        print("❌ Failed to setup H2 database.")
        print("💡 Make sure you ran: python setup_h2.py")
        return
    
    # Get user input
    job_title = "developer"
    location = "Canada" 
    pages = 2
    
    print(f"\n🎯 Scraping '{job_title}' jobs in '{location}' ({pages} pages)...")
    
    # Scrape and save to H2 database
    start_time = time.time()
    jobs = scraper.scrape_jobs(
        job_title=job_title,
        location=location,
        max_pages=pages
    )
    end_time = time.time()
    
    # Display results
    if jobs:
        print(f"\n✅ Successfully processed {len(jobs)} jobs in {end_time - start_time:.1f} seconds")
        
        # Get H2 database statistics
        stats = scraper.get_h2_database_stats()
        print(f"\n📊 H2 DATABASE STATISTICS:")
        print(f"   • Total jobs in H2: {stats.get('total_jobs', 0)}")
        print(f"   • Unique companies: {stats.get('unique_companies', 0)}")
        print(f"   • Unique locations: {stats.get('unique_locations', 0)}")
        
    else:
        print("❌ No jobs were scraped")
    
    print(f"\n💾 H2 database file: indeed_jobs_db.mv.db")
    print("🔧 You can use H2 Console to view the database:")

if __name__ == "__main__":
    main()

🚀 Indeed Scraper with H2 Database
🔧 Using H2 JAR: h2-2.2.224.jar
🔗 Connecting to H2 database: jdbc:h2:file:./indeed_jobs_db;DB_CLOSE_DELAY=-1
❌ Error setting up H2 database: Class org.h2.Driver is not found
❌ Failed to setup H2 database.
💡 Make sure you ran: python setup_h2.py


Traceback (most recent call last):
  File "C:\Users\lesha\AppData\Local\Temp\ipykernel_24228\700885912.py", line 57, in setup_h2_database
    self.db_connection = jaydebeapi.connect(
                         ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\lesha\anaconda3\Lib\site-packages\jaydebeapi\__init__.py", line 412, in connect
    jconn = _jdbc_connect(jclassname, url, driver_args, jars, libs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\lesha\anaconda3\Lib\site-packages\jaydebeapi\__init__.py", line 221, in _jdbc_connect_jpype
    jpype.JClass(jclassname)
  File "C:\Users\lesha\anaconda3\Lib\site-packages\jpype\_jclass.py", line 99, in __new__
    return _jpype._getClass(jc)
           ^^^^^^^^^^^^^^^^^^^^
TypeError: Class org.h2.Driver is not found


In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import random
import re
import sqlite3  # Using SQLite instead of H2 for simplicity
import os
from datetime import datetime
import requests
import zipfile

class IndeedScraperWithDatabase:
    def __init__(self, headless=True):
        self.options = Options()
        
        self.options.add_argument('--no-sandbox')
        self.options.add_argument('--disable-dev-shm-usage')
        self.options.add_argument('--window-size=1920,1080')
        self.options.add_argument('--disable-blink-features=AutomationControlled')
        
        self.options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
        
        if headless:
            self.options.add_argument('--headless')
        
        self.driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()),
            options=self.options
        )
        
        self.jobs_data = []
        self.db_connection = None
        
    def setup_sqlite_database(self, db_path="indeed_jobs.sqlite"):
        """Setup SQLite database connection (no external dependencies needed)"""
        try:
            self.db_connection = sqlite3.connect(db_path)
            self._create_jobs_table()
            print(f"✅ SQLite database connected: {db_path}")
            return True
            
        except Exception as e:
            print(f"❌ Error setting up SQLite database: {e}")
            return False

    def _create_jobs_table(self):
        """Create the jobs table if it doesn't exist"""
        create_table_sql = """
        CREATE TABLE IF NOT EXISTS jobs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT NOT NULL,
            company TEXT,
            location TEXT,
            salary TEXT,
            job_type TEXT,
            link TEXT,
            scraped_date TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            UNIQUE(title, company, location)
        )
        """
        
        cursor = self.db_connection.cursor()
        cursor.execute(create_table_sql)
        self.db_connection.commit()
        cursor.close()
        print("✅ Jobs table created/verified")

    def scrape_jobs(self, job_title="developer", location="Canada", max_pages=3):
        """Scrape jobs and automatically save to database"""
        
        try:
            for page in range(max_pages):
                if page == 0:
                    url = f"https://ca.indeed.com/jobs?q={job_title.replace(' ', '+')}&l={location.replace(' ', '+').replace(',', '%2C')}"
                else:
                    url = f"https://ca.indeed.com/jobs?q={job_title.replace(' ', '+')}&l={location.replace(' ', '+').replace(',', '%2C')}&start={page * 10}"
                
                print(f"Scraping page {page + 1}...")
                
                self.driver.get(url)
                time.sleep(random.uniform(3, 5))
                
                # Wait for job cards
                try:
                    WebDriverWait(self.driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.job_seen_beacon'))
                    )
                except:
                    print(f"No job cards found on page {page + 1}")
                    continue
                
                jobs_count_before = len(self.jobs_data)
                self._extract_page_jobs_css()
                jobs_added = len(self.jobs_data) - jobs_count_before
                
                print(f"Added {jobs_added} jobs from page {page + 1}")
                
                # Save to database after each page
                if jobs_added > 0 and self.db_connection:
                    saved_count = self.save_to_database()
                    print(f"💾 Saved {saved_count} jobs to database")
                
                if page < max_pages - 1:
                    time.sleep(random.uniform(2, 4))
                    
        except Exception as e:
            print(f"Error during scraping: {e}")
        
        finally:
            self.driver.quit()
            if self.db_connection:
                self.db_connection.close()
                print("✅ Database connection closed")
        
        return self.jobs_data

    def _extract_page_jobs_css(self):
        """Extract jobs using reliable CSS selectors"""
        try:
            job_cards = self.driver.find_elements(By.CSS_SELECTOR, 'div.job_seen_beacon')
            print(f"Found {len(job_cards)} job cards")
            
            for card in job_cards:
                job_info = self._extract_job_info_css(card)
                if job_info:
                    self.jobs_data.append(job_info)
                    
        except Exception as e:
            print(f"Error extracting jobs: {e}")

    def _extract_job_info_css(self, card):
        """Extract job information using CSS selectors"""
        try:
            # Title
            title = self._safe_extract(card, 'h2.jobTitle', 'text')
            if title:
                title = re.sub(r'^New\s*', '', title).strip()
            
            # Company
            company = self._safe_extract(card, '[data-testid="company-name"]', 'text')
            if not company:
                company = self._safe_extract(card, '[class*="companyName"]', 'text')
            
            # Location
            location = self._safe_extract(card, '[data-testid="text-location"]', 'text')
            if not location:
                location = self._safe_extract(card, '[class*="companyLocation"]', 'text')
            
            # Salary
            salary = "Salary not specified"
            salary_text = self._safe_extract(card, '[data-testid="attribute_snippet_testid"]', 'text')
            if salary_text and '$' in salary_text:
                salary = salary_text
            
            # Link
            link = self._safe_extract(card, 'h2.jobTitle a', 'href')
            if link and link.startswith('/'):
                link = 'https://ca.indeed.com' + link
            
            # Job Type
            job_type = self._extract_job_type(card)
            
            if title:
                return {
                    'title': title,
                    'company': company or "Company not found",
                    'location': location or "Location not specified",
                    'salary': salary,
                    'job_type': job_type,
                    'link': link or "Link not available",
                    'scraped_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                }
                
        except Exception as e:
            print(f"Error extracting job info: {e}")
        
        return None

    def _safe_extract(self, parent, selector, attribute='text'):
        """Safely extract element text or attribute"""
        try:
            element = parent.find_element(By.CSS_SELECTOR, selector)
            if attribute == 'text':
                return element.text.strip()
            else:
                return element.get_attribute(attribute)
        except:
            return None

    def _extract_job_type(self, card):
        """Extract job type"""
        try:
            card_text = card.text.lower()
            if 'full-time' in card_text:
                return 'Full-time'
            elif 'part-time' in card_text:
                return 'Part-time'
            elif 'contract' in card_text:
                return 'Contract'
            elif 'temporary' in card_text:
                return 'Temporary'
            elif 'remote' in card_text:
                return 'Remote'
            else:
                return 'Not specified'
        except:
            return 'Not specified'

    def save_to_database(self):
        """Save current jobs data to database"""
        if not self.jobs_data or not self.db_connection:
            print("No data to save or database not connected")
            return 0
        
        try:
            cursor = self.db_connection.cursor()
            saved_count = 0
            
            insert_sql = """
            INSERT OR IGNORE INTO jobs (title, company, location, salary, job_type, link, scraped_date)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            """
            
            for job in self.jobs_data:
                try:
                    cursor.execute(insert_sql, (
                        job['title'],
                        job['company'],
                        job['location'],
                        job['salary'],
                        job['job_type'],
                        job['link'],
                        job['scraped_date']
                    ))
                    if cursor.rowcount > 0:
                        saved_count += 1
                except Exception as e:
                    print(f"Error inserting job: {e}")
                    continue
            
            self.db_connection.commit()
            cursor.close()
            return saved_count
            
        except Exception as e:
            print(f"Error saving to database: {e}")
            return 0

    def query_jobs_from_database(self, limit=10):
        """Query jobs from database"""
        if not self.db_connection:
            print("Database not connected")
            return []
        
        try:
            # Reconnect if needed
            if not os.path.exists("indeed_jobs.sqlite"):
                return []
                
            conn = sqlite3.connect("indeed_jobs.sqlite")
            cursor = conn.cursor()
            
            cursor.execute("""
                SELECT title, company, location, salary, job_type, link, scraped_date 
                FROM jobs 
                ORDER BY scraped_date DESC 
                LIMIT ?
            """, (limit,))
            
            results = cursor.fetchall()
            cursor.close()
            conn.close()
            
            jobs = []
            for row in results:
                jobs.append({
                    'title': row[0],
                    'company': row[1],
                    'location': row[2],
                    'salary': row[3],
                    'job_type': row[4],
                    'link': row[5],
                    'scraped_date': row[6]
                })
            
            return jobs
            
        except Exception as e:
            print(f"Error querying database: {e}")
            return []

    def get_database_stats(self):
        """Get statistics from the database"""
        if not os.path.exists("indeed_jobs.sqlite"):
            return {}
        
        try:
            conn = sqlite3.connect("indeed_jobs.sqlite")
            cursor = conn.cursor()
            
            # Total jobs count
            cursor.execute("SELECT COUNT(*) FROM jobs")
            total_jobs = cursor.fetchone()[0]
            
            # Companies count
            cursor.execute("SELECT COUNT(DISTINCT company) FROM jobs")
            unique_companies = cursor.fetchone()[0]
            
            # Locations count
            cursor.execute("SELECT COUNT(DISTINCT location) FROM jobs")
            unique_locations = cursor.fetchone()[0]
            
            # Latest scrape date
            cursor.execute("SELECT MAX(scraped_date) FROM jobs")
            latest_scrape = cursor.fetchone()[0]
            
            cursor.close()
            conn.close()
            
            return {
                'total_jobs': total_jobs,
                'unique_companies': unique_companies,
                'unique_locations': unique_locations,
                'latest_scrape': latest_scrape
            }
            
        except Exception as e:
            print(f"Error getting database stats: {e}")
            return {}

    def save_to_csv(self, filename="indeed_jobs_backup.csv"):
        """Backup database to CSV file"""
        if not os.path.exists("indeed_jobs.sqlite"):
            print("Database file not found")
            return False
        
        try:
            conn = sqlite3.connect("indeed_jobs.sqlite")
            cursor = conn.cursor()
            cursor.execute("SELECT * FROM jobs")
            
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
            
            df = pd.DataFrame(data, columns=columns)
            df.to_csv(filename, index=False)
            
            cursor.close()
            conn.close()
            print(f"💾 Database backed up to {filename}")
            return True
            
        except Exception as e:
            print(f"Error backing up to CSV: {e}")
            return False

    def export_to_excel(self, filename="indeed_jobs.xlsx"):
        """Export database to Excel file"""
        if not os.path.exists("indeed_jobs.sqlite"):
            print("Database file not found")
            return False
        
        try:
            conn = sqlite3.connect("indeed_jobs.sqlite")
            df = pd.read_sql_query("SELECT * FROM jobs", conn)
            df.to_excel(filename, index=False)
            conn.close()
            print(f"💾 Database exported to {filename}")
            return True
            
        except Exception as e:
            print(f"Error exporting to Excel: {e}")
            return False

def main():
    print("🚀 Indeed Scraper with SQLite Database")
    print("=" * 50)
    
    # Initialize scraper
    scraper = IndeedScraperWithDatabase(headless=True)
    
    # Setup SQLite database (no external dependencies needed)
    if not scraper.setup_sqlite_database("indeed_jobs.sqlite"):
        print("Failed to setup database. Exiting.")
        return
    
    # Get user input
    job_title = input("Enter job title (default: developer): ").strip() or "developer"
    location = input("Enter location (default: Canada): ").strip() or "Canada"
    pages = input("Enter number of pages to scrape (default: 2): ").strip()
    pages = int(pages) if pages.isdigit() else 2
    
    print(f"\nScraping '{job_title}' jobs in '{location}' ({pages} pages)...")
    
    # Scrape and save to database
    start_time = time.time()
    jobs = scraper.scrape_jobs(
        job_title=job_title,
        location=location,
        max_pages=pages
    )
    end_time = time.time()
    
    # Display results
    if jobs:
        print(f"\n✅ Successfully processed {len(jobs)} jobs in {end_time - start_time:.1f} seconds")
        
        # Get database statistics
        stats = scraper.get_database_stats()
        print(f"\n📊 DATABASE STATISTICS:")
        print(f"   • Total jobs in database: {stats.get('total_jobs', 0)}")
        print(f"   • Unique companies: {stats.get('unique_companies', 0)}")
        print(f"   • Unique locations: {stats.get('unique_locations', 0)}")
        print(f"   • Latest scrape: {stats.get('latest_scrape', 'N/A')}")
        
        # Query and display recent jobs from database
        recent_jobs = scraper.query_jobs_from_database(limit=5)
        if recent_jobs:
            print(f"\n📋 RECENT JOBS FROM DATABASE:")
            for i, job in enumerate(recent_jobs, 1):
                print(f"\n{i}. {job['title']}")
                print(f"   Company: {job['company']}")
                print(f"   Location: {job['location']}")
                print(f"   Salary: {job['salary']}")
                print(f"   Scraped: {job['scraped_date']}")
        
        # Backup to CSV and Excel
        scraper.save_to_csv("indeed_jobs_backup.csv")
        scraper.export_to_excel("indeed_jobs.xlsx")
        
    else:
        print("❌ No jobs were scraped")
    
    print(f"\n💾 Database file: indeed_jobs.sqlite")
    print("💾 CSV backup: indeed_jobs_backup.csv")
    print("💾 Excel export: indeed_jobs.xlsx")
    print("\n🔧 You can open the SQLite database with:")
    print("   - DB Browser for SQLite (GUI)")
    print("   - sqlite3 command line tool")
    print("   - Any SQL client that supports SQLite")

# Function to query the database later
def query_database_later():
    """Function to query the database after scraping is complete"""
    scraper = IndeedScraperWithDa
    tabase()
    jobs = scraper.query_jobs_from_database(limit=50)
    stats = scraper.get_database_stats()
    
    print(f"📊 Current database stats:")
    print(f"   Total jobs: {stats.get('total_jobs', 0)}")
    print(f"   Unique companies: {stats.get('unique_companies', 0)}")
    
    return jobs, stats

if __name__ == "__main__":
    main()

🚀 Indeed Scraper with SQLite Database
✅ Jobs table created/verified
✅ SQLite database connected: indeed_jobs.sqlite


KeyboardInterrupt: Interrupted by user

In [12]:
# Query the database anytime
jobs, stats = query_database_later()

# Or use DB Browser for SQLite (free GUI tool) to browse the data

Database not connected
📊 Current database stats:
   Total jobs: 33
   Unique companies: 30


In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import random
import re
import sqlite3
import os
from datetime import datetime

class IndeedScraperWithDatabase:
    def __init__(self, headless=True):
        self.options = Options()
        
        self.options.add_argument('--no-sandbox')
        self.options.add_argument('--disable-dev-shm-usage')
        self.options.add_argument('--window-size=1920,1080')
        self.options.add_argument('--disable-blink-features=AutomationControlled')
        
        self.options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
        
        if headless:
            self.options.add_argument('--headless')
        
        self.driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()),
            options=self.options
        )
        
        self.jobs_data = []
        self.db_connection = None
        
    def setup_sqlite_database(self, db_path="indeed_jobs.sqlite"):
        try:
            self.db_connection = sqlite3.connect(db_path)
            self._create_jobs_table()
            print(f"SQLite database connected: {db_path}")
            return True
            
        except Exception as e:
            print(f"Error setting up SQLite database: {e}")
            return False

    def _create_jobs_table(self):

        create_table_sql = """
        CREATE TABLE IF NOT EXISTS jobs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT NOT NULL,
            company TEXT,
            location TEXT,
            salary TEXT,
            job_type TEXT,
            link TEXT,
            scraped_date TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            UNIQUE(title, company, location)
        )
        """
        
        cursor = self.db_connection.cursor()
        cursor.execute(create_table_sql)
        self.db_connection.commit()
        cursor.close()
        print("Jobs table created/verified")

    def scrape_jobs(self, job_title="", location="Canada", max_pages=5):
        
        try:
            for page in range(max_pages):
                if page == 0:
                    url = f"https://ca.indeed.com/jobs?q={job_title.replace(' ', '+')}&l={location.replace(' ', '+').replace(',', '%2C')}"
                else:
                    url = f"https://ca.indeed.com/jobs?q={job_title.replace(' ', '+')}&l={location.replace(' ', '+').replace(',', '%2C')}&start={page * 10}"
                
                print(f"Scraping page {page + 1}...")
                print(f"URL: {url}")
                
                self.driver.get(url)
                time.sleep(random.uniform(3, 5))
                
                # Wait for job cards
                try:
                    WebDriverWait(self.driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.job_seen_beacon'))
                    )
                except:
                    print(f"No job cards found on page {page + 1}")
                    continue
                
                jobs_count_before = len(self.jobs_data)
                self._extract_page_jobs_css()
                jobs_added = len(self.jobs_data) - jobs_count_before
                
                print(f"Added {jobs_added} jobs from page {page + 1}")
                
                # Save to database after each page
                if jobs_added > 0 and self.db_connection:
                    saved_count = self.save_to_database()
                    print(f"Saved {saved_count} jobs to database")
                
                if page < max_pages - 1:
                    time.sleep(random.uniform(2, 4))
                    
        except Exception as e:
            print(f"Error during scraping: {e}")
        
        finally:
            self.driver.quit()
            if self.db_connection:
                self.db_connection.close()
                print("Database connection closed")
        
        return self.jobs_data

    def _extract_page_jobs_css(self):
        try:
            job_cards = self.driver.find_elements(By.CSS_SELECTOR, 'div.job_seen_beacon')
            print(f"Found {len(job_cards)} job cards")
            
            for card in job_cards:
                job_info = self._extract_job_info_css(card)
                if job_info:
                    self.jobs_data.append(job_info)
                    
        except Exception as e:
            print(f"Error extracting jobs: {e}")

    def _extract_job_info_css(self, card):
        try:
            # Title
            title = self._safe_extract(card, 'h2.jobTitle', 'text')
            if title:
                title = re.sub(r'^New\s*', '', title).strip()
            
            # Company
            company = self._safe_extract(card, '[data-testid="company-name"]', 'text')
            if not company:
                company = self._safe_extract(card, '[class*="companyName"]', 'text')
            
            # Location
            location = self._safe_extract(card, '[data-testid="text-location"]', 'text')
            if not location:
                location = self._safe_extract(card, '[class*="companyLocation"]', 'text')
            
            # Salary
            salary = "Salary not specified"
            salary_text = self._safe_extract(card, '[data-testid="attribute_snippet_testid"]', 'text')
            if salary_text and '$' in salary_text:
                salary = salary_text
            
            # Link
            link = self._safe_extract(card, 'h2.jobTitle a', 'href')
            if link and link.startswith('/'):
                link = 'https://ca.indeed.com' + link
            
            # Job Type
            job_type = self._extract_job_type(card)
            
            if title:
                return {
                    'title': title,
                    'company': company or "Company not found",
                    'location': location or "Location not specified",
                    'salary': salary,
                    'job_type': job_type,
                    'link': link or "Link not available",
                    'scraped_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                }
                
        except Exception as e:
            print(f"Error extracting job info: {e}")
        
        return None

    def _safe_extract(self, parent, selector, attribute='text'):
        try:
            element = parent.find_element(By.CSS_SELECTOR, selector)
            if attribute == 'text':
                return element.text.strip()
            else:
                return element.get_attribute(attribute)
        except:
            return None

    def _extract_job_type(self, card):
        try:
            card_text = card.text.lower()
            if 'full-time' in card_text:
                return 'Full-time'
            elif 'part-time' in card_text:
                return 'Part-time'
            elif 'contract' in card_text:
                return 'Contract'
            elif 'temporary' in card_text:
                return 'Temporary'
            elif 'remote' in card_text:
                return 'Remote'
            else:
                return 'Not specified'
        except:
            return 'Not specified'

    def save_to_database(self):
        if not self.jobs_data or not self.db_connection:
            print("No data to save or database not connected")
            return 0
        
        try:
            cursor = self.db_connection.cursor()
            saved_count = 0
            
            insert_sql = """
            INSERT OR IGNORE INTO jobs (title, company, location, salary, job_type, link, scraped_date)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            """
            
            for job in self.jobs_data:
                try:
                    cursor.execute(insert_sql, (
                        job['title'],
                        job['company'],
                        job['location'],
                        job['salary'],
                        job['job_type'],
                        job['link'],
                        job['scraped_date']
                    ))
                    if cursor.rowcount > 0:
                        saved_count += 1
                except Exception as e:
                    print(f"Error inserting job: {e}")
                    continue
            
            self.db_connection.commit()
            cursor.close()
            return saved_count
            
        except Exception as e:
            print(f"Error saving to database: {e}")
            return 0

    def query_jobs_from_database(self, limit=10):
        if not self.db_connection:
            print("Database not connected")
            return []
        
        try:
            # Reconnect if needed
            if not os.path.exists("indeed_jobs.sqlite"):
                return []
                
            conn = sqlite3.connect("indeed_jobs.sqlite")
            cursor = conn.cursor()
            
            cursor.execute("""
                SELECT title, company, location, salary, job_type, link, scraped_date 
                FROM jobs 
                ORDER BY scraped_date DESC 
                LIMIT ?
            """, (limit,))
            
            results = cursor.fetchall()
            cursor.close()
            conn.close()
            
            jobs = []
            for row in results:
                jobs.append({
                    'title': row[0],
                    'company': row[1],
                    'location': row[2],
                    'salary': row[3],
                    'job_type': row[4],
                    'link': row[5],
                    'scraped_date': row[6]
                })
            
            return jobs
            
        except Exception as e:
            print(f"Error querying database: {e}")
            return []

    def get_database_stats(self):
        if not os.path.exists("indeed_jobs.sqlite"):
            return {}
        
        try:
            conn = sqlite3.connect("indeed_jobs.sqlite")
            cursor = conn.cursor()
            
            # Total jobs count
            cursor.execute("SELECT COUNT(*) FROM jobs")
            total_jobs = cursor.fetchone()[0]
            
            # Companies count
            cursor.execute("SELECT COUNT(DISTINCT company) FROM jobs")
            unique_companies = cursor.fetchone()[0]
            
            # Locations count
            cursor.execute("SELECT COUNT(DISTINCT location) FROM jobs")
            unique_locations = cursor.fetchone()[0]
            
            # Latest scrape date
            cursor.execute("SELECT MAX(scraped_date) FROM jobs")
            latest_scrape = cursor.fetchone()[0]
            
            cursor.close()
            conn.close()
            
            return {
                'total_jobs': total_jobs,
                'unique_companies': unique_companies,
                'unique_locations': unique_locations,
                'latest_scrape': latest_scrape
            }
            
        except Exception as e:
            print(f"Error getting database stats: {e}")
            return {}

    def save_to_csv(self, filename="indeed_jobs_backup.csv"):
        if not os.path.exists("indeed_jobs.sqlite"):
            print("Database file not found")
            return False
        
        try:
            conn = sqlite3.connect("indeed_jobs.sqlite")
            cursor = conn.cursor()
            cursor.execute("SELECT * FROM jobs")
            
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
            
            df = pd.DataFrame(data, columns=columns)
            df.to_csv(filename, index=False)
            
            cursor.close()
            conn.close()
            print(f"Database backed up to {filename}")
            return True
            
        except Exception as e:
            print(f"Error backing up to CSV: {e}")
            return False

    def export_to_excel(self, filename="indeed_jobs.xlsx"):
        if not os.path.exists("indeed_jobs.sqlite"):
            print("Database file not found")
            return False
        
        try:
            conn = sqlite3.connect("indeed_jobs.sqlite")
            df = pd.read_sql_query("SELECT * FROM jobs", conn)
            df.to_excel(filename, index=False)
            conn.close()
            print(f"Database exported to {filename}")
            return True
            
        except Exception as e:
            print(f"Error exporting to Excel: {e}")
            return False

def main():
    
    # Initialize scraper
    scraper = IndeedScraperWithDatabase(headless=True)
    
    # Setup SQLite database
    if not scraper.setup_sqlite_database("indeed_jobs.sqlite"):
        print("Failed to setup database. Exiting.")
        return
    
    job_title = ""  
    location = "Canada"
    pages = 5
    
    print(f"\nScraping all jobs in '{location}' ({pages} pages)...")
    
    # Scrape and save to database
    start_time = time.time()
    jobs = scraper.scrape_jobs(
        job_title=job_title,
        location=location,
        max_pages=pages
    )
    end_time = time.time()
    
    # Display results
    if jobs:
        print(f"\nSuccessfully processed {len(jobs)} jobs in {end_time - start_time:.1f} seconds")
        
        # Get database statistics
        stats = scraper.get_database_stats()
        print(f"\nDATABASE STATISTICS:")
        print(f"   • Total jobs in database: {stats.get('total_jobs', 0)}")
        print(f"   • Unique companies: {stats.get('unique_companies', 0)}")
        print(f"   • Unique locations: {stats.get('unique_locations', 0)}")
        print(f"   • Latest scrape: {stats.get('latest_scrape', 'N/A')}")
        
        # Query and display recent jobs from database
        recent_jobs = scraper.query_jobs_from_database(limit=5)
        if recent_jobs:
            print(f"\nRECENT JOBS FROM DATABASE:")
            for i, job in enumerate(recent_jobs, 1):
                print(f"\n{i}. {job['title']}")
                print(f"   Company: {job['company']}")
                print(f"   Location: {job['location']}")
                print(f"   Salary: {job['salary']}")
                print(f"   Scraped: {job['scraped_date']}")
        
        # Backup to CSV
        scraper.save_to_csv("indeed_jobs_backup.csv")
        
    else:
        print("No jobs were scraped")
    
    print(f"\nDatabase file: indeed_jobs.sqlite")
    print("CSV backup: indeed_jobs_backup.csv")

def query_database_later():
    scraper = IndeedScraperWithDatabase()
    jobs = scraper.query_jobs_from_database(limit=50)
    stats = scraper.get_database_stats()
    
    print(f"Current database stats:")
    print(f"   Total jobs: {stats.get('total_jobs', 0)}")
    print(f"   Unique companies: {stats.get('unique_companies', 0)}")
    
    return jobs, stats

if __name__ == "__main__":
    main()

Jobs table created/verified
SQLite database connected: indeed_jobs.sqlite

Scraping all jobs in 'Canada' (5 pages)...
Scraping page 1...
URL: https://ca.indeed.com/jobs?q=&l=Canada
Found 16 job cards
Added 15 jobs from page 1
💾 Saved 5 jobs to database
Scraping page 2...
URL: https://ca.indeed.com/jobs?q=&l=Canada&start=10
Found 16 job cards
Added 15 jobs from page 2
💾 Saved 6 jobs to database
Scraping page 3...
URL: https://ca.indeed.com/jobs?q=&l=Canada&start=20
Found 16 job cards
Added 15 jobs from page 3
💾 Saved 4 jobs to database
Scraping page 4...
URL: https://ca.indeed.com/jobs?q=&l=Canada&start=30
Found 16 job cards
Added 15 jobs from page 4
💾 Saved 3 jobs to database
Scraping page 5...
URL: https://ca.indeed.com/jobs?q=&l=Canada&start=40
Found 16 job cards
Added 15 jobs from page 5
💾 Saved 3 jobs to database
Database connection closed

Successfully processed 75 jobs in 51.5 seconds

DATABASE STATISTICS:
   • Total jobs in database: 90
   • Unique companies: 78
   • Unique loca

In [17]:
# Query the database anytime
jobs, stats = query_database_later()

# Or use DB Browser for SQLite (free GUI tool) to browse the data

Database not connected
Current database stats:
   Total jobs: 90
   Unique companies: 78


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import random
import re
import sqlite3
import os
from datetime import datetime

class IndeedScraperWithDatabase:
    def __init__(self, headless=True):
        self.options = Options()
        
        self.options.add_argument('--no-sandbox')
        self.options.add_argument('--disable-dev-shm-usage')
        self.options.add_argument('--window-size=1920,1080')
        self.options.add_argument('--disable-blink-features=AutomationControlled')
        
        self.options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
        
        if headless:
            self.options.add_argument('--headless')
        
        self.driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()),
            options=self.options
        )
        
        self.jobs_data = []
        self.db_connection = None
        
    def setup_sqlite_database(self, db_path="indeed_jobs.sqlite"):
        try:
            self.db_connection = sqlite3.connect(db_path)
            self._create_jobs_table()
            print(f"SQLite database connected: {db_path}")
            return True
            
        except Exception as e:
            print(f"Error setting up SQLite database: {e}")
            return False

    def _create_jobs_table(self):
        create_table_sql = """
        CREATE TABLE IF NOT EXISTS jobs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT NOT NULL,
            company TEXT,
            location TEXT,
            salary TEXT,
            job_type TEXT,
            link TEXT,
            description TEXT,
            scraped_date TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            UNIQUE(title, company, location)
        )
        """
        
        cursor = self.db_connection.cursor()
        cursor.execute(create_table_sql)
        self.db_connection.commit()
        cursor.close()
        print("Jobs table created/verified")

    def scrape_jobs(self, job_title="", location="Canada", max_pages=5):
        try:
            for page in range(max_pages):
                if page == 0:
                    url = f"https://ca.indeed.com/jobs?q={job_title.replace(' ', '+')}&l={location.replace(' ', '+').replace(',', '%2C')}"
                else:
                    url = f"https://ca.indeed.com/jobs?q={job_title.replace(' ', '+')}&l={location.replace(' ', '+').replace(',', '%2C')}&start={page * 10}"
                
                print(f"Scraping page {page + 1}...")
                print(f"URL: {url}")
                
                self.driver.get(url)
                time.sleep(random.uniform(3, 5))
                
                # Wait for job cards
                try:
                    WebDriverWait(self.driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.job_seen_beacon'))
                    )
                except:
                    print(f"No job cards found on page {page + 1}")
                    continue
                
                jobs_count_before = len(self.jobs_data)
                self._extract_page_jobs_css()
                jobs_added = len(self.jobs_data) - jobs_count_before
                
                print(f"Added {jobs_added} jobs from page {page + 1}")
                
                # Save to database after each page
                if jobs_added > 0 and self.db_connection:
                    saved_count = self.save_to_database()
                    print(f"Saved {saved_count} jobs to database")
                
                if page < max_pages - 1:
                    time.sleep(random.uniform(2, 4))
                    
        except Exception as e:
            print(f"Error during scraping: {e}")
        
        finally:
            self.driver.quit()
            if self.db_connection:
                self.db_connection.close()
                print("Database connection closed")
        
        return self.jobs_data

    def _extract_page_jobs_css(self):
        try:
            job_cards = self.driver.find_elements(By.CSS_SELECTOR, 'div.job_seen_beacon')
            print(f"Found {len(job_cards)} job cards")
            
            for i, card in enumerate(job_cards):
                # Add delay to avoid being blocked
                if i > 0 and i % 3 == 0:
                    time.sleep(1)
                
                job_info = self._extract_job_info_css(card)
                if job_info:
                    self.jobs_data.append(job_info)
                    
        except Exception as e:
            print(f"Error extracting jobs: {e}")

    def _extract_job_info_css(self, card):
        try:
            # Title
            title = self._safe_extract(card, 'h2.jobTitle', 'text')
            if title:
                title = re.sub(r'^New\s*', '', title).strip()
            
            # Company
            company = self._safe_extract(card, '[data-testid="company-name"]', 'text')
            if not company:
                company = self._safe_extract(card, '[class*="companyName"]', 'text')
            
            # Location
            location = self._safe_extract(card, '[data-testid="text-location"]', 'text')
            if not location:
                location = self._safe_extract(card, '[class*="companyLocation"]', 'text')
            
            # Salary
            salary = "Salary not specified"
            salary_text = self._safe_extract(card, '[data-testid="attribute_snippet_testid"]', 'text')
            if salary_text and '$' in salary_text:
                salary = salary_text
            
            # Link
            link = self._safe_extract(card, 'h2.jobTitle a', 'href')
            if link and link.startswith('/'):
                link = 'https://ca.indeed.com' + link
            
            # Job Type
            job_type = self._extract_job_type(card)
            
            # Job Description - Extract from snippet on main page
            description = self._extract_job_description(card)
            
            if title:
                return {
                    'title': title,
                    'company': company or "Company not found",
                    'location': location or "Location not specified",
                    'salary': salary,
                    'job_type': job_type,
                    'link': link or "Link not available",
                    'description': description or "Description not available",
                    'scraped_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                }
                
        except Exception as e:
            print(f"Error extracting job info: {e}")
        
        return None

    def _extract_job_description(self, card):
        """Extract job description from the job card snippet"""
        try:
            # Try multiple selectors for job description snippets
            description_selectors = [
                'div[class*="job-snippet"]',
                'div[class*="description-snippet"]',
                'div[class*="summary"]',
                '.job-snippet',
                '.job-snippet ul',
                'div[class*="metadata"]'
            ]
            
            for selector in description_selectors:
                description = self._safe_extract(card, selector, 'text')
                if description and len(description.strip()) > 20:  # Minimum length to be meaningful
                    # Clean up the description
                    description = description.strip()
                    # Remove extra whitespace and newlines
                    description = re.sub(r'\s+', ' ', description)
                    return description[:1000]  # Limit to 1000 characters
            
            # If no specific description element found, try to get key details from the card
            card_text = card.text
            lines = [line.strip() for line in card_text.split('\n') if line.strip()]
            
            # Look for description-like text (usually appears after company and location)
            description_keywords = ['responsibilities', 'requirements', 'qualifications', 'experience', 'skills', 'duties']
            
            for i, line in enumerate(lines):
                line_lower = line.lower()
                # Skip if it's title, company, location, salary, or job type
                if (any(keyword in line_lower for keyword in description_keywords) or
                    (len(line) > 50 and not any(indicator in line_lower for indicator in 
                     ['$', 'salary', 'full-time', 'part-time', 'contract', 'remote', 'temporary']))):
                    return line[:500]  # Return first 500 characters
            
            return "Description snippet not available"
            
        except Exception as e:
            print(f"Error extracting description: {e}")
            return "Error extracting description"

    def _safe_extract(self, parent, selector, attribute='text'):
        try:
            element = parent.find_element(By.CSS_SELECTOR, selector)
            if attribute == 'text':
                return element.text.strip()
            else:
                return element.get_attribute(attribute)
        except:
            return None

    def _extract_job_type(self, card):
        try:
            card_text = card.text.lower()
            if 'full-time' in card_text:
                return 'Full-time'
            elif 'part-time' in card_text:
                return 'Part-time'
            elif 'contract' in card_text:
                return 'Contract'
            elif 'temporary' in card_text:
                return 'Temporary'
            elif 'remote' in card_text:
                return 'Remote'
            else:
                return 'Not specified'
        except:
            return 'Not specified'

    def save_to_database(self):
        if not self.jobs_data or not self.db_connection:
            print("No data to save or database not connected")
            return 0
        
        try:
            cursor = self.db_connection.cursor()
            saved_count = 0
            
            insert_sql = """
            INSERT OR IGNORE INTO jobs (title, company, location, salary, job_type, link, description, scraped_date)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            """
            
            for job in self.jobs_data:
                try:
                    cursor.execute(insert_sql, (
                        job['title'],
                        job['company'],
                        job['location'],
                        job['salary'],
                        job['job_type'],
                        job['link'],
                        job['description'],
                        job['scraped_date']
                    ))
                    if cursor.rowcount > 0:
                        saved_count += 1
                except Exception as e:
                    print(f"Error inserting job: {e}")
                    continue
            
            self.db_connection.commit()
            cursor.close()
            return saved_count
            
        except Exception as e:
            print(f"Error saving to database: {e}")
            return 0

    def query_jobs_from_database(self, limit=10):
        if not self.db_connection:
            print("Database not connected")
            return []
        
        try:
            # Reconnect if needed
            if not os.path.exists("indeed_jobs.sqlite"):
                return []
                
            conn = sqlite3.connect("indeed_jobs.sqlite")
            cursor = conn.cursor()
            
            cursor.execute("""
                SELECT title, company, location, salary, job_type, link, description, scraped_date 
                FROM jobs 
                ORDER BY scraped_date DESC 
                LIMIT ?
            """, (limit,))
            
            results = cursor.fetchall()
            cursor.close()
            conn.close()
            
            jobs = []
            for row in results:
                jobs.append({
                    'title': row[0],
                    'company': row[1],
                    'location': row[2],
                    'salary': row[3],
                    'job_type': row[4],
                    'link': row[5],
                    'description': row[6],
                    'scraped_date': row[7]
                })
            
            return jobs
            
        except Exception as e:
            print(f"Error querying database: {e}")
            return []

    def get_database_stats(self):
        if not os.path.exists("indeed_jobs.sqlite"):
            return {}
        
        try:
            conn = sqlite3.connect("indeed_jobs.sqlite")
            cursor = conn.cursor()
            
            # Total jobs count
            cursor.execute("SELECT COUNT(*) FROM jobs")
            total_jobs = cursor.fetchone()[0]
            
            # Companies count
            cursor.execute("SELECT COUNT(DISTINCT company) FROM jobs")
            unique_companies = cursor.fetchone()[0]
            
            # Locations count
            cursor.execute("SELECT COUNT(DISTINCT location) FROM jobs")
            unique_locations = cursor.fetchone()[0]
            
            # Jobs with descriptions
            cursor.execute("SELECT COUNT(*) FROM jobs WHERE description != 'Description snippet not available'")
            jobs_with_descriptions = cursor.fetchone()[0]
            
            # Latest scrape date
            cursor.execute("SELECT MAX(scraped_date) FROM jobs")
            latest_scrape = cursor.fetchone()[0]
            
            cursor.close()
            conn.close()
            
            return {
                'total_jobs': total_jobs,
                'unique_companies': unique_companies,
                'unique_locations': unique_locations,
                'jobs_with_descriptions': jobs_with_descriptions,
                'latest_scrape': latest_scrape
            }
            
        except Exception as e:
            print(f"Error getting database stats: {e}")
            return {}

    def save_to_csv(self, filename="indeed_jobs_backup.csv"):
        if not os.path.exists("indeed_jobs.sqlite"):
            print("Database file not found")
            return False
        
        try:
            conn = sqlite3.connect("indeed_jobs.sqlite")
            cursor = conn.cursor()
            cursor.execute("SELECT * FROM jobs")
            
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
            
            df = pd.DataFrame(data, columns=columns)
            df.to_csv(filename, index=False, encoding='utf-8')
            
            cursor.close()
            conn.close()
            print(f"Database backed up to {filename}")
            return True
            
        except Exception as e:
            print(f"Error backing up to CSV: {e}")
            return False

    def export_to_excel(self, filename="indeed_jobs.xlsx"):
        if not os.path.exists("indeed_jobs.sqlite"):
            print("Database file not found")
            return False
        
        try:
            conn = sqlite3.connect("indeed_jobs.sqlite")
            df = pd.read_sql_query("SELECT * FROM jobs", conn)
            df.to_excel(filename, index=False)
            conn.close()
            print(f"Database exported to {filename}")
            return True
            
        except Exception as e:
            print(f"Error exporting to Excel: {e}")
            return False

def main():
    # Initialize scraper
    scraper = IndeedScraperWithDatabase(headless=True)
    
    # Setup SQLite database
    if not scraper.setup_sqlite_database("indeed_jobs.sqlite"):
        print("Failed to setup database. Exiting.")
        return
    
    job_title = ""  
    location = "Canada"
    pages = 5
    
    print(f"\nScraping all jobs in '{location}' ({pages} pages)...")
    
    # Scrape and save to database
    start_time = time.time()
    jobs = scraper.scrape_jobs(
        job_title=job_title,
        location=location,
        max_pages=pages
    )
    end_time = time.time()
    
    # Display results
    if jobs:
        print(f"\nSuccessfully processed {len(jobs)} jobs in {end_time - start_time:.1f} seconds")
        
        # Get database statistics
        stats = scraper.get_database_stats()
        print(f"\nDATABASE STATISTICS:")
        print(f"   • Total jobs in database: {stats.get('total_jobs', 0)}")
        print(f"   • Unique companies: {stats.get('unique_companies', 0)}")
        print(f"   • Unique locations: {stats.get('unique_locations', 0)}")
        print(f"   • Jobs with descriptions: {stats.get('jobs_with_descriptions', 0)}")
        print(f"   • Latest scrape: {stats.get('latest_scrape', 'N/A')}")
        
        # Query and display recent jobs from database
        recent_jobs = scraper.query_jobs_from_database(limit=3)
        if recent_jobs:
            print(f"\nRECENT JOBS FROM DATABASE:")
            for i, job in enumerate(recent_jobs, 1):
                print(f"\n{i}. {job['title']}")
                print(f"   Company: {job['company']}")
                print(f"   Location: {job['location']}")
                print(f"   Salary: {job['salary']}")
                print(f"   Type: {job['job_type']}")
                print(f"   Description: {job['description'][:150]}...")
                print(f"   Scraped: {job['scraped_date']}")
        
        # Backup to CSV
        scraper.save_to_csv("indeed_jobs_backup.csv")
        
    else:
        print("No jobs were scraped")
    
    print(f"\nDatabase file: indeed_jobs.sqlite")
    print("CSV backup: indeed_jobs_backup.csv")

def query_database_later():
    scraper = IndeedScraperWithDatabase()
    jobs = scraper.query_jobs_from_database(limit=50)
    stats = scraper.get_database_stats()
    
    print(f"Current database stats:")
    print(f"   Total jobs: {stats.get('total_jobs', 0)}")
    print(f"   Unique companies: {stats.get('unique_companies', 0)}")
    print(f"   Jobs with descriptions: {stats.get('jobs_with_descriptions', 0)}")
    
    return jobs, stats

if __name__ == "__main__":
    main()

Jobs table created/verified
SQLite database connected: indeed_jobs.sqlite

Scraping all jobs in 'Canada' (5 pages)...
Scraping page 1...
URL: https://ca.indeed.com/jobs?q=&l=Canada
Found 16 job cards
Added 15 jobs from page 1
Saved 14 jobs to database
Scraping page 2...
URL: https://ca.indeed.com/jobs?q=&l=Canada&start=10
Found 16 job cards
Added 15 jobs from page 2
Saved 15 jobs to database
Scraping page 3...
URL: https://ca.indeed.com/jobs?q=&l=Canada&start=20
Found 16 job cards
Added 15 jobs from page 3
Saved 12 jobs to database
Scraping page 4...
URL: https://ca.indeed.com/jobs?q=&l=Canada&start=30
Found 16 job cards
Added 15 jobs from page 4
Saved 13 jobs to database
Scraping page 5...
URL: https://ca.indeed.com/jobs?q=&l=Canada&start=40
Found 16 job cards
Added 15 jobs from page 5
Saved 10 jobs to database
Database connection closed

Successfully processed 75 jobs in 88.4 seconds

DATABASE STATISTICS:
   • Total jobs in database: 64
   • Unique companies: 62
   • Unique locations

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import random
import re
import sqlite3
import os
from datetime import datetime

class ImprovedIndeedScraper:
    def __init__(self, headless=True):
        self.options = Options()
        
        self.options.add_argument('--no-sandbox')
        self.options.add_argument('--disable-dev-shm-usage')
        self.options.add_argument('--window-size=1920,1080')
        self.options.add_argument('--disable-blink-features=AutomationControlled')
        self.options.add_argument('--disable-gpu')
        self.options.add_argument('--disable-extensions')
        
        self.options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
        
        if headless:
            self.options.add_argument('--headless')
        
        self.driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()),
            options=self.options
        )
        
        self.jobs_data = []
        self.db_connection = None
        
    def setup_sqlite_database(self, db_path="indeed_jobs.sqlite"):
        try:
            self.db_connection = sqlite3.connect(db_path)
            self._create_jobs_table()
            print(f"SQLite database connected: {db_path}")
            return True
            
        except Exception as e:
            print(f"Error setting up SQLite database: {e}")
            return False

    def _create_jobs_table(self):
        create_table_sql = """
        CREATE TABLE IF NOT EXISTS jobs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT NOT NULL,
            company TEXT,
            location TEXT,
            salary TEXT,
            job_type TEXT,
            link TEXT,
            description TEXT,
            scraped_date TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            UNIQUE(title, company, location)
        )
        """
        
        cursor = self.db_connection.cursor()
        cursor.execute(create_table_sql)
        self.db_connection.commit()
        cursor.close()
        print("Jobs table created/verified")

    def scrape_jobs(self, job_title="", location="Canada", max_pages=5):
        try:
            for page in range(max_pages):
                if page == 0:
                    url = f"https://ca.indeed.com/jobs?q={job_title.replace(' ', '+')}&l={location.replace(' ', '+').replace(',', '%2C')}"
                else:
                    url = f"https://ca.indeed.com/jobs?q={job_title.replace(' ', '+')}&l={location.replace(' ', '+').replace(',', '%2C')}&start={page * 10}"
                
                print(f"Scraping page {page + 1}...")
                print(f"URL: {url}")
                
                self.driver.get(url)
                time.sleep(random.uniform(3, 5))
                
                # Wait for job cards
                try:
                    WebDriverWait(self.driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.job_seen_beacon'))
                    )
                except:
                    print(f"No job cards found on page {page + 1}")
                    continue
                
                # Handle cookie consent if present
                self._handle_cookie_consent()
                
                jobs_count_before = len(self.jobs_data)
                self._extract_page_jobs_improved()
                jobs_added = len(self.jobs_data) - jobs_count_before
                
                print(f"Added {jobs_added} jobs from page {page + 1}")
                
                # Save to database after each page
                if jobs_added > 0 and self.db_connection:
                    saved_count = self.save_to_database()
                    print(f"Saved {saved_count} jobs to database")
                
                if page < max_pages - 1:
                    time.sleep(random.uniform(2, 4))
                    
        except Exception as e:
            print(f"Error during scraping: {e}")
        
        finally:
            self.driver.quit()
            if self.db_connection:
                self.db_connection.close()
                print("Database connection closed")
        
        return self.jobs_data

    def _handle_cookie_consent(self):
        """Handle cookie consent popup if it appears"""
        try:
            cookie_selectors = [
                'button[aria-label="reject"]',
                'button[aria-label="Reject All"]',
                'button#onetrust-reject-all-handler',
                'button[data-testid="reject-button"]'
            ]
            
            for selector in cookie_selectors:
                try:
                    reject_button = WebDriverWait(self.driver, 2).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, selector))
                    )
                    reject_button.click()
                    print("Cookie consent handled")
                    time.sleep(1)
                    break
                except:
                    continue
        except:
            pass

    def _extract_page_jobs_improved(self):
        try:
            job_cards = self.driver.find_elements(By.CSS_SELECTOR, 'div.job_seen_beacon')
            print(f"Found {len(job_cards)} job cards")
            
            for i, card in enumerate(job_cards):
                # Add delay to avoid being blocked
                if i > 0 and i % 3 == 0:
                    time.sleep(random.uniform(1, 2))
                
                job_info = self._extract_job_info_improved(card)
                if job_info:
                    self.jobs_data.append(job_info)
                    
        except Exception as e:
            print(f"Error extracting jobs: {e}")

    def _extract_job_info_improved(self, card):
        try:
            # Extract basic info first
            title = self._safe_extract(card, 'h2.jobTitle a', 'text')
            if not title:
                title = self._safe_extract(card, 'h2 a', 'text')
            if title:
                title = re.sub(r'^New\s*', '', title).strip()
            
            company = self._safe_extract(card, '[data-testid="company-name"]', 'text')
            if not company:
                company = self._safe_extract(card, '.companyName', 'text')
            if not company:
                company = self._safe_extract(card, '[class*="companyName"]', 'text')
            
            location = self._safe_extract(card, '[data-testid="text-location"]', 'text')
            if not location:
                location = self._safe_extract(card, '.companyLocation', 'text')
            if not location:
                location = self._safe_extract(card, '[class*="companyLocation"]', 'text')
            
            # Now extract description using the basic info we have
            description = self._extract_job_description_improved(card, title, company, location)
            
            # Salary - Improved extraction
            salary = "Salary not specified"
            salary_selectors = [
                '[data-testid="attribute_snippet_testid"]',
                '.salary-snippet-container',
                '.metadata salary-snippet-container'
            ]
            
            for selector in salary_selectors:
                salary_text = self._safe_extract(card, selector, 'text')
                if salary_text and ('$' in salary_text or 'hour' in salary_text.lower() or 'year' in salary_text.lower()):
                    salary = salary_text
                    break
            
            # Link
            link = self._safe_extract(card, 'h2.jobTitle a', 'href')
            if not link:
                link = self._safe_extract(card, 'h2 a', 'href')
            if link and link.startswith('/'):
                link = 'https://ca.indeed.com' + link
            
            # Job Type - Improved extraction
            job_type = self._extract_job_type_improved(card)
            
            if title:
                return {
                    'title': title,
                    'company': company or "Company not found",
                    'location': location or "Location not specified",
                    'salary': salary,
                    'job_type': job_type,
                    'link': link or "Link not available",
                    'description': description or "Description not available",
                    'scraped_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                }
                
        except Exception as e:
            print(f"Error extracting job info: {e}")
        
        return None

    def _extract_job_description_improved(self, card, title, company, location):
        """Improved job description extraction with proper parameters"""
        try:
            # Try the job snippet/summary section first
            description_selectors = [
                'div.job-snippet',
                'div[class*="job-snippet"]',
                'div[class*="snippet"]',
                'div[class*="summary"]',
                'ul[style*="list-style-type:circle"]',
                '.css-e9ucx3',  # From the HTML structure
                '[data-testid="belowJobSnippet"]'
            ]
            
            for selector in description_selectors:
                description = self._safe_extract(card, selector, 'text')
                if description and len(description.strip()) > 20:
                    description = description.strip()
                    description = re.sub(r'\s+', ' ', description)
                    return description[:1000]  # Limit to 1000 characters
            
            # Try to extract from the entire card text with better filtering
            card_text = card.text
            if card_text:
                lines = [line.strip() for line in card_text.split('\n') if line.strip()]
                
                # Look for description content after basic info
                basic_info_indicators = ['$', 'salary', 'full-time', 'part-time', 'contract', 
                                       'remote', 'temporary', 'permanent', 'urgently hiring',
                                       'apply', 'save', 'bookmark', 'easily apply']
                
                description_lines = []
                
                for line in lines:
                    line_lower = line.lower()
                    
                    # Skip if it's basic info we've already captured
                    if any(indicator in line_lower for indicator in basic_info_indicators):
                        continue
                    
                    # Skip if it's title, company, or location (usually shorter lines that match exactly)
                    if (title and title.lower() in line_lower) or \
                       (company and company.lower() in line_lower) or \
                       (location and location.lower() in line_lower):
                        continue
                    
                    # Skip very short lines (likely navigation or buttons)
                    if len(line) < 20:
                        continue
                        
                    # If we find a reasonably long line that's not basic info, consider it description
                    if len(line) > 30:
                        description_lines.append(line)
                
                if description_lines:
                    description = ' '.join(description_lines[:3])  # Take first 3 description lines
                    description = re.sub(r'\s+', ' ', description).strip()
                    if len(description) > 30:
                        return description[:800]
            
            return "Description snippet not available"
            
        except Exception as e:
            print(f"Error extracting description: {e}")
            return "Error extracting description"

    def _extract_job_type_improved(self, card):
        """Improved job type extraction"""
        try:
            card_text = card.text.lower()
            
            # Check for job type in metadata
            metadata_selectors = [
                '[data-testid="attribute_snippet_testid"]',
                '.metadata',
                '.css-5ooe72'
            ]
            
            for selector in metadata_selectors:
                try:
                    elements = card.find_elements(By.CSS_SELECTOR, selector)
                    for element in elements:
                        text = element.text.lower()
                        if 'full-time' in text:
                            return 'Full-time'
                        elif 'part-time' in text:
                            return 'Part-time'
                        elif 'contract' in text:
                            return 'Contract'
                        elif 'temporary' in text:
                            return 'Temporary'
                        elif 'permanent' in text:
                            return 'Permanent'
                        elif 'remote' in text:
                            return 'Remote'
                except:
                    continue
            
            # Fallback to text analysis
            if 'full-time' in card_text:
                return 'Full-time'
            elif 'part-time' in card_text:
                return 'Part-time'
            elif 'contract' in card_text:
                return 'Contract'
            elif 'temporary' in card_text:
                return 'Temporary'
            elif 'remote' in card_text:
                return 'Remote'
            else:
                return 'Not specified'
                
        except:
            return 'Not specified'

    def _safe_extract(self, parent, selector, attribute='text'):
        try:
            element = parent.find_element(By.CSS_SELECTOR, selector)
            if attribute == 'text':
                return element.text.strip()
            else:
                return element.get_attribute(attribute)
        except:
            return None

    def save_to_database(self):
        if not self.jobs_data or not self.db_connection:
            print("No data to save or database not connected")
            return 0
        
        try:
            cursor = self.db_connection.cursor()
            saved_count = 0
            
            insert_sql = """
            INSERT OR IGNORE INTO jobs (title, company, location, salary, job_type, link, description, scraped_date)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            """
            
            for job in self.jobs_data:
                try:
                    cursor.execute(insert_sql, (
                        job['title'],
                        job['company'],
                        job['location'],
                        job['salary'],
                        job['job_type'],
                        job['link'],
                        job['description'],
                        job['scraped_date']
                    ))
                    if cursor.rowcount > 0:
                        saved_count += 1
                except Exception as e:
                    print(f"Error inserting job: {e}")
                    continue
            
            self.db_connection.commit()
            cursor.close()
            return saved_count
            
        except Exception as e:
            print(f"Error saving to database: {e}")
            return 0

    def query_jobs_from_database(self, limit=10):
        if not self.db_connection:
            print("Database not connected")
            return []
        
        try:
            # Reconnect if needed
            if not os.path.exists("indeed_jobs.sqlite"):
                return []
                
            conn = sqlite3.connect("indeed_jobs.sqlite")
            cursor = conn.cursor()
            
            cursor.execute("""
                SELECT title, company, location, salary, job_type, link, description, scraped_date 
                FROM jobs 
                ORDER BY scraped_date DESC 
                LIMIT ?
            """, (limit,))
            
            results = cursor.fetchall()
            cursor.close()
            conn.close()
            
            jobs = []
            for row in results:
                jobs.append({
                    'title': row[0],
                    'company': row[1],
                    'location': row[2],
                    'salary': row[3],
                    'job_type': row[4],
                    'link': row[5],
                    'description': row[6],
                    'scraped_date': row[7]
                })
            
            return jobs
            
        except Exception as e:
            print(f"Error querying database: {e}")
            return []

    def get_database_stats(self):
        if not os.path.exists("indeed_jobs.sqlite"):
            return {}
        
        try:
            conn = sqlite3.connect("indeed_jobs.sqlite")
            cursor = conn.cursor()
            
            # Total jobs count
            cursor.execute("SELECT COUNT(*) FROM jobs")
            total_jobs = cursor.fetchone()[0]
            
            # Companies count
            cursor.execute("SELECT COUNT(DISTINCT company) FROM jobs")
            unique_companies = cursor.fetchone()[0]
            
            # Locations count
            cursor.execute("SELECT COUNT(DISTINCT location) FROM jobs")
            unique_locations = cursor.fetchone()[0]
            
            # Jobs with descriptions
            cursor.execute("SELECT COUNT(*) FROM jobs WHERE description != 'Description snippet not available' AND description != 'Error extracting description'")
            jobs_with_descriptions = cursor.fetchone()[0]
            
            # Latest scrape date
            cursor.execute("SELECT MAX(scraped_date) FROM jobs")
            latest_scrape = cursor.fetchone()[0]
            
            cursor.close()
            conn.close()
            
            return {
                'total_jobs': total_jobs,
                'unique_companies': unique_companies,
                'unique_locations': unique_locations,
                'jobs_with_descriptions': jobs_with_descriptions,
                'latest_scrape': latest_scrape
            }
            
        except Exception as e:
            print(f"Error getting database stats: {e}")
            return {}

    def save_to_csv(self, filename="indeed_jobs_backup.csv"):
        if not os.path.exists("indeed_jobs.sqlite"):
            print("Database file not found")
            return False
        
        try:
            conn = sqlite3.connect("indeed_jobs.sqlite")
            cursor = conn.cursor()
            cursor.execute("SELECT * FROM jobs")
            
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
            
            df = pd.DataFrame(data, columns=columns)
            df.to_csv(filename, index=False, encoding='utf-8')
            
            cursor.close()
            conn.close()
            print(f"Database backed up to {filename}")
            return True
            
        except Exception as e:
            print(f"Error backing up to CSV: {e}")
            return False

    def export_to_excel(self, filename="indeed_jobs.xlsx"):
        if not os.path.exists("indeed_jobs.sqlite"):
            print("Database file not found")
            return False
        
        try:
            conn = sqlite3.connect("indeed_jobs.sqlite")
            df = pd.read_sql_query("SELECT * FROM jobs", conn)
            df.to_excel(filename, index=False)
            conn.close()
            print(f"Database exported to {filename}")
            return True
            
        except Exception as e:
            print(f"Error exporting to Excel: {e}")
            return False

def main():
    # Initialize scraper
    scraper = ImprovedIndeedScraper(headless=True)
    
    # Setup SQLite database
    if not scraper.setup_sqlite_database("indeed_jobs.sqlite"):
        print("Failed to setup database. Exiting.")
        return
    
    job_title = ""  
    location = "Canada"
    pages = 3  # Start with fewer pages for testing
    
    print(f"\nScraping all jobs in '{location}' ({pages} pages)...")
    
    # Scrape and save to database
    start_time = time.time()
    jobs = scraper.scrape_jobs(
        job_title=job_title,
        location=location,
        max_pages=pages
    )
    end_time = time.time()
    
    # Display results
    if jobs:
        print(f"\nSuccessfully processed {len(jobs)} jobs in {end_time - start_time:.1f} seconds")

        # Backup to CSV
        scraper.save_to_csv("indeed_jobs_backup.csv")
        
    else:
        print("No jobs were scraped")
    
    print(f"\nDatabase file: indeed_jobs.sqlite")
    print("CSV backup: indeed_jobs_backup.csv")

if __name__ == "__main__":
    main()

Jobs table created/verified
SQLite database connected: indeed_jobs.sqlite

Scraping all jobs in 'Canada' (3 pages)...
Scraping page 1...
URL: https://ca.indeed.com/jobs?q=&l=Canada
Found 16 job cards
Added 15 jobs from page 1
Saved 4 jobs to database
Scraping page 2...
URL: https://ca.indeed.com/jobs?q=&l=Canada&start=10
No job cards found on page 2
Scraping page 3...
URL: https://ca.indeed.com/jobs?q=&l=Canada&start=20
Found 16 job cards
Added 15 jobs from page 3
Saved 5 jobs to database
Database connection closed

Successfully processed 30 jobs in 76.5 seconds

SAMPLE JOBS SCRAPED:

1. Invetory Counter
   Company: Bannons Gas Bar
   Location: Thunder Bay, ON P7J 1K5
   Salary: $17.20–$20.00 an hour
   Type: Full-time
   Description: Description snippet not available...

2. Housekeeper/Cleaner
   Company: Genie Clean
   Location: Williams Lake, BC
   Salary: $21.31–$22.03 an hour
   Type: Full-time
   Description: Description snippet not available...

3. Office Assistant/Receptionist
